In [1]:

import datetime
import numpy as np
import cv2
from itertools import cycle
import pickle
import pathlib
import math
import tqdm
import scipy.io
from matplotlib import pyplot as plt
import scipy.io
import h5py
import re
from lxml import etree as ET
import scipy.signal as sig
import pandas as pd
from scipy.stats import kde
from BlockSync_current import BlockSync
import UtilityFunctions_newOE as uf
from scipy import signal
import bokeh
import seaborn as sns
from matplotlib import rcParams
%matplotlib inline
plt.style.use('default')
rcParams['pdf.fonttype'] = 42  # Ensure fonts are embedded and editable
rcParams['ps.fonttype'] = 42  # Ensure compatibility with vector outputs


def bokeh_plotter(data_list, x_axis_list=None, label_list=None,
                  plot_name='default',
                  x_axis_label='X', y_axis_label='Y',
                  peaks=None, peaks_list=False, export_path=False):
    """Generates an interactive Bokeh plot for the given data vector.
    Args:
        data_list (list or array): The data to be plotted.
        label_list (list of str): The labels of the data vectors
        plot_name (str, optional): The title of the plot. Defaults to 'default'.
        x_axis (str, optional): The label for the x-axis. Defaults to 'X'.
        y_axis (str, optional): The label for the y-axis. Defaults to 'Y'.
        peaks (list or array, optional): Indices of peaks to highlight on the plot. Defaults to None.
        export_path (False or str): when set to str, will output the resulting html fig
    """
    color_cycle = cycle(bokeh.palettes.Category10_10)
    fig = bokeh.plotting.figure(title=f'bokeh explorer: {plot_name}',
                                x_axis_label=x_axis_label,
                                y_axis_label=y_axis_label,
                                plot_width=1500,
                                plot_height=700)

    for i, data_vector in enumerate(data_list):

        color = next(color_cycle)

        if x_axis_list is None:
            x_axis = range(len(data_vector))
        elif len(x_axis_list) == len(data_list):
            print('x_axis manually set')
            x_axis = x_axis_list[i]
        else:
            raise Exception(
                'problem with x_axis_list input - should be either None, or a list with the same length as data_list')
        if label_list is None:
            fig.line(x_axis, data_vector, line_color=color, legend_label=f"Line {i + 1}")
        elif len(label_list) == len(data_list):
            fig.line(range(len(data_vector)), data_vector, line_color=color, legend_label=f"{label_list[i]}")
        if peaks is not None and peaks_list is True:
            fig.circle(peaks[i], data_vector[peaks[i]], size=10, color=color)

    if peaks is not None and peaks_list is False:
        fig.circle(peaks, data_vector[peaks], size=10, color='red')

    if export_path is not False:
        print(f'exporting to {export_path}')
        bokeh.io.output.output_file(filename=str(export_path / f'{plot_name}.html'), title=f'{plot_name}')
    bokeh.plotting.show(fig)


def load_eye_data_2d_w_rotation_matrix(block):
    """
    This function checks if the eye dataframes and rotation dict object exist, then imports them
    :param block: The current blocksync class with verifiec re/le dfs
    :return: None
    """
    try:
        block.left_eye_data = pd.read_csv(block.analysis_path / 'left_eye_data.csv', index_col=0, engine='python')
        block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data.csv', index_col=0, engine='python')
    except FileNotFoundError:
        print('eye_data files not found, run the pipeline!')
        return

    try:
        with open(block.analysis_path / 'rotate_eye_data_params.pkl', 'rb') as file:
            rotation_dict = pickle.load(file)
            block.left_rotation_matrix = rotation_dict['left_rotation_matrix']
            block.right_rotation_matrix = rotation_dict['right_rotation_matrix']
            block.left_rotation_angle = rotation_dict['left_rotation_angle']
            block.right_rotation_angle = rotation_dict['right_rotation_angle']
    except FileNotFoundError:
        print('No rotation matrix file, create it')


def create_saccade_events_df(eye_data_df, speed_threshold, bokeh_verify_threshold=False, magnitude_calib=1,
                             speed_profile=True):
    """
    Detects saccade events in eye tracking data and computes relevant metrics.

    Parameters:
    - eye_data_df (pd.DataFrame): Input DataFrame containing eye tracking data.
    - speed_threshold (float): Threshold for saccade detection based on speed.

    Returns:
    - df (pd.DataFrame): Modified input DataFrame with added columns for speed and saccade detection.
    - saccade_events_df (pd.DataFrame): DataFrame containing information about detected saccade events.

    Steps:
    1. Calculate speed components ('speed_x', 'speed_y') based on differences in 'center_x' and 'center_y'.
    2. Compute the magnitude of the velocity vector ('speed_r').
    3. Create a binary column ('is_saccade') indicating saccade events based on the speed threshold.
    4. Determine saccade onset and offset indices and timestamps.
    5. Create a DataFrame ('saccade_events_df') with columns:
        - 'saccade_start_ind': Indices of saccade onset.
        - 'saccade_start_timestamp': Timestamps corresponding to saccade onset.
        - 'saccade_end_ind': Indices of saccade offset.
        - 'saccade_end_timestamp': Timestamps corresponding to saccade offset.
        - 'length': Duration of each saccade event.
    6. Calculate distance traveled and angles for each saccade event.
    7. Append additional columns to 'saccade_events_df':
        - 'magnitude': Magnitude of the distance traveled during each saccade.
        - 'angle': Angle of the saccade vector in degrees.
        - 'initial_x', 'initial_y': Initial coordinates of the saccade.
        - 'end_x', 'end_y': End coordinates of the saccade.

    Note: The original 'eye_data_df' is not modified; modified data is returned as 'df'.
    """
    df = eye_data_df
    df['speed_x'] = df['center_x'].diff()  # Difference between consecutive 'center_x' values
    df['speed_y'] = df['center_y'].diff()  # Difference between consecutive 'center_y' values

    # Step 2: Calculate magnitude of the velocity vector (R vector speed)
    df['speed_r'] = (df['speed_x'] ** 2 + df['speed_y'] ** 2) ** 0.5

    # Create a column for saccade detection
    df['is_saccade'] = df['speed_r'] > speed_threshold

    # create a saccade_on_off indicator where 1 is rising edge and -1 is falling edge by subtracting a shifted binary mask
    saccade_on_off = df.is_saccade.astype(int) - df.is_saccade.shift(periods=1, fill_value=False).astype(int)
    saccade_on_inds = np.where(saccade_on_off == 1)[
                          0] - 1  # notice the manual shift here, chosen to include the first (sometimes slower) eye frame, just before saccade threshold crossing
    saccade_on_ms = df['ms_axis'].iloc[saccade_on_inds]
    saccade_on_timestamps = df['OE_timestamp'].iloc[saccade_on_inds]
    saccade_off_inds = np.where(saccade_on_off == -1)[0]
    saccade_off_timestamps = df['OE_timestamp'].iloc[saccade_off_inds]
    saccade_off_ms = df['ms_axis'].iloc[saccade_off_inds]

    saccade_dict = {'saccade_start_ind': saccade_on_inds,
                    'saccade_start_timestamp': saccade_on_timestamps.values,
                    'saccade_end_ind': saccade_off_inds,
                    'saccade_end_timestamp': saccade_off_timestamps.values,
                    'saccade_on_ms': saccade_on_ms.values,
                    'saccade_off_ms': saccade_off_ms.values}

    saccade_events_df = pd.DataFrame.from_dict(saccade_dict)
    saccade_events_df['length'] = saccade_events_df['saccade_end_ind'] - saccade_events_df['saccade_start_ind']
    # Drop columns used for intermediate steps
    df = df.drop(['is_saccade'], axis=1)

    distances = []
    angles = []
    speed_list = []
    diameter_list = []
    for index, row in tqdm.tqdm(saccade_events_df.iterrows()):
        saccade_samples = df.loc[(df['OE_timestamp'] >= row['saccade_start_timestamp']) &
                                 (df['OE_timestamp'] <= row['saccade_end_timestamp'])]
        distance_traveled = saccade_samples['speed_r'].sum()
        if speed_profile:
            saccade_speed_profile = saccade_samples['speed_r'].values
            speed_list.append(saccade_speed_profile)
        saccade_diameter_profile = saccade_samples['pupil_diameter'].values
        diameter_list.append(saccade_diameter_profile)
        # Calculate angle from initial position to endpoint
        initial_position = saccade_samples.iloc[0][['center_x', 'center_y']]
        endpoint = saccade_samples.iloc[-1][['center_x', 'center_y']]
        overall_angle = np.arctan2(endpoint['center_y'] - initial_position['center_y'],
                                   endpoint['center_x'] - initial_position['center_x'])

        angles.append(overall_angle)
        distances.append(distance_traveled)

    saccade_events_df['magnitude_raw'] = np.array(distances)
    saccade_events_df['magnitude'] = np.array(distances) * magnitude_calib
    saccade_events_df['angle'] = np.where(np.isnan(angles), angles, np.rad2deg(
        angles) % 360)  # Convert radians to degrees and ensure result is in [0, 360)
    start_ts = saccade_events_df['saccade_start_timestamp'].values
    end_ts = saccade_events_df['saccade_end_timestamp'].values
    saccade_start_df = df[df['OE_timestamp'].isin(start_ts)]
    saccade_end_df = df[df['OE_timestamp'].isin(end_ts)]
    start_x_coord = saccade_start_df['center_x']
    start_y_coord = saccade_start_df['center_y']
    end_x_coord = saccade_end_df['center_x']
    end_y_coord = saccade_end_df['center_y']
    saccade_events_df['initial_x'] = start_x_coord.values
    saccade_events_df['initial_y'] = start_y_coord.values
    saccade_events_df['end_x'] = end_x_coord.values
    saccade_events_df['end_y'] = end_y_coord.values
    saccade_events_df['calib_dx'] = (saccade_events_df['end_x'].values - saccade_events_df[
        'initial_x'].values) * magnitude_calib
    saccade_events_df['calib_dy'] = (saccade_events_df['end_y'].values - saccade_events_df[
        'initial_y'].values) * magnitude_calib
    if speed_profile:
        saccade_events_df['speed_profile'] = speed_list
    saccade_events_df['diameter_profile'] = diameter_list
    if bokeh_verify_threshold:
        bokeh_plotter(data_list=[df.speed_r], label_list=['Pupil Velocity'], peaks=saccade_on_inds)

    return df, saccade_events_df


# create a multi-animal block_collection:

def create_block_collections(animals, block_lists, experiment_path, bad_blocks=None):
    """
    Create block collections and a block dictionary from multiple animals and their respective block lists.

    Parameters:
    - animals: list of str, names of the animals.
    - block_lists: list of lists of int, block numbers corresponding to each animal.
    - experiment_path: pathlib.Path, path to the experiment directory.
    - bad_blocks: list of int, blocks to exclude. Default is an empty list.

    Returns:
    - block_collection: list of BlockSync objects for all specified blocks.
    - block_dict: dictionary where keys are block numbers as strings and values are BlockSync objects.
    """
    import UtilityFunctions_newOE as uf

    if bad_blocks is None:
        bad_blocks = []

    block_collection = []
    block_dict = {}

    for animal, blocks in zip(animals, block_lists):
        # Generate blocks for the current animal
        current_blocks = uf.block_generator(
            block_numbers=blocks,
            experiment_path=experiment_path,
            animal=animal,
            bad_blocks=bad_blocks
        )
        # Add to collection and dictionary
        block_collection.extend(current_blocks)
        for b in current_blocks:
            block_dict[f"{animal}_block_{b.block_num}"] = b

    return block_collection, block_dict


In [2]:
# Example usage:
# This was the previous run
#animals = ['PV_62', 'PV_126', 'PV_57']
#block_lists = [[24, 26, 38], [7, 8, 9, 10, 11, 12], [7, 8, 9, 12, 13]]
#This with new animals:
animals = ['PV_106','PV_143','PV_62','PV_126', 'PV_57']
block_lists = [[8,9,10,11,12],[1,2,3,4],[24, 26, 38],[7, 8, 9, 10, 11, 12], [7, 8, 9, 12, 13]]
experiment_path = pathlib.Path(r"Z:\Nimrod\experiments")
bad_blocks = [0]  # Example of bad blocks

block_collection, block_dict = create_block_collections(
    animals=animals,
    block_lists=block_lists,
    experiment_path=experiment_path,
    bad_blocks=bad_blocks
)
for block in block_collection:
    print(f'working on {block}')
    block.parse_open_ephys_events()
    block.get_eye_brightness_vectors()
    block.synchronize_block()
    block.create_eye_brightness_df(threshold_value=20)

    # if the code fails here, go to manual synchronization
    #block.import_manual_sync_df()
    block.read_dlc_data()
    block.calibrate_pixel_size(10)
    #load_eye_data_2d_w_rotation_matrix(block) #should be integrated again... later

instantiated block number 008 at Path: Z:\Nimrod\experiments\PV_106\2025_08_06\block_008, new OE version
Found the sample rate for block 008 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 008
got it!
instantiated block number 009 at Path: Z:\Nimrod\experiments\PV_106\2025_08_06\block_009, new OE version
Found the sample rate for block 009 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 009
got it!
instantiated block number 010 at Path: Z:\Nimrod\experiments\PV_106\2025_08_06\block_010, new OE version
Found the sample rate for block 010 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 010
got it!
instantiated block number 011 at Path: Z:\Nimrod\exp

In [3]:
    for block in block_collection:
        print('working on {}'.format(block))
        # block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_corr_angles.csv')
        # block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_corr_angles.csv')
        #block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_degrees_raw_xflipped.csv')
        #block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_raw_xflipped.csv')
        block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_degrees_raw_verified.csv')
        block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_raw_verified.csv')
        # block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_3d_corr_verified.csv')
        # block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_3d_corr_verified.csv')
        #block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_degrees_rotated_verified.csv')
        #block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_rotated_verified.csv')

    # calibrate pupil diameter:
    # if 'pupil_diameter' not in block.left_eye_data.columns:
    #     block.left_eye_data['pupil_diameter_pixels'] = block.left_eye_data.major_ax * 2 * np.pi
    #     block.right_eye_data['pupil_diameter_pixels'] = block.right_eye_data.major_ax * 2 * np.pi
    #     block.left_eye_data['pupil_diameter'] = block.left_eye_data['pupil_diameter_pixels'] * block.L_pix_size
    #     block.right_eye_data['pupil_diameter'] = block.right_eye_data['pupil_diameter_pixels'] * block.R_pix_size

working on PV_106, block 008, on PV106_ET_d3t12025-08-06_11-52-19
working on PV_106, block 009, on PV106_ET_d3t2_2025-08-06_12-09-43
working on PV_106, block 010, on PV106_ET_d3t3_2025-08-06_12-26-43
working on PV_106, block 011, on PV106_ET_d3t4_2025-08-06_12-44-30
working on PV_106, block 012, on PV106_ET_d3t5_2025-08-06_13-21-30
working on PV_143, block 001, on PV143_ET_d1t1_2025-08-11_13-29-08
working on PV_143, block 002, on PV143_ET_d1t2_2025-08-11_13-50-11
working on PV_143, block 003, on PV143_ET_d1t3_2025-08-11_14-20-35
working on PV_143, block 004, on PV143_ET_d1t4_2025-08-11_14-58-28
working on PV_62, block 024, on 2023-04-27_11-22-56
working on PV_62, block 026, on 2023-04-27_12-21-41
working on PV_62, block 038, on 2023-05-01_13-57-45
working on PV_126, block 007, on PV126_Trial16_wake3_2024-07-18_12-49-12
working on PV_126, block 008, on PV126_Trial16_wake4_2024-07-18_13-24-41
working on PV_126, block 009, on PV126_Trial18_wake5_2024-07-18_14-39-15
working on PV_126, bloc

In [4]:
block_collection

[BlockSync object for animal PV_106 with 
 block_num 008 at date PV106_ET_d3t12025-08-06_11-52-19,
 BlockSync object for animal PV_106 with 
 block_num 009 at date PV106_ET_d3t2_2025-08-06_12-09-43,
 BlockSync object for animal PV_106 with 
 block_num 010 at date PV106_ET_d3t3_2025-08-06_12-26-43,
 BlockSync object for animal PV_106 with 
 block_num 011 at date PV106_ET_d3t4_2025-08-06_12-44-30,
 BlockSync object for animal PV_106 with 
 block_num 012 at date PV106_ET_d3t5_2025-08-06_13-21-30,
 BlockSync object for animal PV_143 with 
 block_num 001 at date PV143_ET_d1t1_2025-08-11_13-29-08,
 BlockSync object for animal PV_143 with 
 block_num 002 at date PV143_ET_d1t2_2025-08-11_13-50-11,
 BlockSync object for animal PV_143 with 
 block_num 003 at date PV143_ET_d1t3_2025-08-11_14-20-35,
 BlockSync object for animal PV_143 with 
 block_num 004 at date PV143_ET_d1t4_2025-08-11_14-58-28,
 BlockSync object for animal PV_62 with 
 block_num 024 at date 2023-04-27_11-22-56,
 BlockSync objec

In [5]:
for block in block_collection:
  if 'pupil_diameter' not in block.left_eye_data.columns:
        print(f'calculating pupil diameter for {block} ')
        block.left_eye_data['pupil_diameter_pixels'] = block.left_eye_data.major_ax
        block.right_eye_data['pupil_diameter_pixels'] = block.right_eye_data.major_ax
        block.left_eye_data['pupil_diameter'] = block.left_eye_data['pupil_diameter_pixels'] * block.L_pix_size
        block.right_eye_data['pupil_diameter'] = block.right_eye_data['pupil_diameter_pixels'] * block.R_pix_size

calculating pupil diameter for PV_106, block 008, on PV106_ET_d3t12025-08-06_11-52-19 
calculating pupil diameter for PV_106, block 009, on PV106_ET_d3t2_2025-08-06_12-09-43 
calculating pupil diameter for PV_106, block 010, on PV106_ET_d3t3_2025-08-06_12-26-43 
calculating pupil diameter for PV_106, block 011, on PV106_ET_d3t4_2025-08-06_12-44-30 
calculating pupil diameter for PV_106, block 012, on PV106_ET_d3t5_2025-08-06_13-21-30 
calculating pupil diameter for PV_143, block 001, on PV143_ET_d1t1_2025-08-11_13-29-08 
calculating pupil diameter for PV_143, block 002, on PV143_ET_d1t2_2025-08-11_13-50-11 
calculating pupil diameter for PV_143, block 003, on PV143_ET_d1t3_2025-08-11_14-20-35 
calculating pupil diameter for PV_143, block 004, on PV143_ET_d1t4_2025-08-11_14-58-28 
calculating pupil diameter for PV_126, block 007, on PV126_Trial16_wake3_2024-07-18_12-49-12 
calculating pupil diameter for PV_126, block 008, on PV126_Trial16_wake4_2024-07-18_13-24-41 
calculating pupil dia

In [6]:
# # uncomment to switch corrected values to k_phi / theta columns
# for block in block_collection:
#     block.left_eye_data.drop(columns=['k_phi', 'k_theta'], inplace=True)
#     block.right_eye_data.drop(columns=['k_phi', 'k_theta'], inplace=True)
#     # then rename in-place
#     block.left_eye_data = block.left_eye_data.rename(columns={'corr_phi': 'k_phi', 'corr_theta': 'k_theta'})
#     block.right_eye_data = block.right_eye_data.rename(columns={'corr_phi': 'k_phi', 'corr_theta': 'k_theta'})

In [7]:
# If you want to recenter the data (for span-related plots) against the median position, uncomment and run this:
import pandas as pd
import numpy as np
from typing import Tuple, Dict

def recenter_eye_angles_to_rest(
    eye_df: pd.DataFrame,
    phi_col: str = "k_phi",
    theta_col: str = "k_theta",
    inplace: bool = False,
    dropna: bool = True,
) -> Tuple[pd.DataFrame, Dict[str, float]]:
    """
    Recenter the eye's angular coordinates so that the median (resting position)
    of k_phi and k_theta becomes zero degrees.

    Parameters
    ----------
    eye_df : pd.DataFrame
        Eye dataframe containing k_phi and k_theta columns (can be *_eye_data_clean).
    phi_col, theta_col : str
        Column names for the vertical (phi) and horizontal (theta) gaze angles.
    inplace : bool
        If True, modifies the input DataFrame in place; otherwise returns a copy.
    dropna : bool
        Whether to ignore NaN values when computing medians (recommended True).

    Returns
    -------
    df_out : pd.DataFrame
        DataFrame with corrected columns (values centered around 0).
    offsets : dict
        {'phi_offset': float, 'theta_offset': float}
        The medians that were subtracted from the original data.

    Notes
    -----
    - New columns 'k_phi_recentered' and 'k_theta_recentered' are added
      (or the originals replaced if inplace=True).
    - This assumes your angular units are degrees.
    - The offsets correspond to the eye's resting orientation in the camera frame.
    """
    # --- validate ---
    for c in (phi_col, theta_col):
        if c not in eye_df.columns:
            raise ValueError(f"Column '{c}' not found in dataframe")

    # Select working DataFrame
    df = eye_df if inplace else eye_df.copy()

    # Convert to numeric, handle NaN
    phi_vals = pd.to_numeric(df[phi_col], errors="coerce")
    theta_vals = pd.to_numeric(df[theta_col], errors="coerce")

    # Compute medians (resting position)
    phi_med = float(np.nanmedian(phi_vals)) if dropna else float(np.median(phi_vals))
    theta_med = float(np.nanmedian(theta_vals)) if dropna else float(np.median(theta_vals))

    # Apply centering
    df[phi_col] = phi_vals - phi_med
    df[theta_col] = theta_vals - theta_med

    # Optionally store explicitly labeled recentered columns
    df[f"{phi_col}_recentered"] = df[phi_col]
    df[f"{theta_col}_recentered"] = df[theta_col]

    offsets = {"phi_offset": phi_med, "theta_offset": theta_med}
    return df, offsets

for block in block_collection:
    # Apply to the cleaned left eye data
    df_left_clean = block.left_eye_data
    df_left_centered, left_offsets = recenter_eye_angles_to_rest(df_left_clean)

    # Same for right
    df_right_clean = block.right_eye_data
    df_right_centered, right_offsets = recenter_eye_angles_to_rest(df_right_clean)

    # Store back on the block if desired
    block.left_eye_data_centered = df_left_centered
    block.right_eye_data_centered = df_right_centered

    print("Left eye offsets:", left_offsets)
    print("Right eye offsets:", right_offsets)


Left eye offsets: {'phi_offset': -13.585779190063475, 'theta_offset': -43.059635162353516}
Right eye offsets: {'phi_offset': 6.654082298278809, 'theta_offset': -42.897987365722656}
Left eye offsets: {'phi_offset': -17.727107048034668, 'theta_offset': -42.58493423461914}
Right eye offsets: {'phi_offset': 27.34492111206055, 'theta_offset': -35.46980285644531}
Left eye offsets: {'phi_offset': -10.522362232208252, 'theta_offset': -43.67327690124512}
Right eye offsets: {'phi_offset': 18.342082977294922, 'theta_offset': -41.04368591308594}
Left eye offsets: {'phi_offset': -17.197317123413086, 'theta_offset': -39.99274444580078}
Right eye offsets: {'phi_offset': 15.23730182647705, 'theta_offset': -35.542009353637695}
Left eye offsets: {'phi_offset': -9.613146305084229, 'theta_offset': -42.25788688659668}
Right eye offsets: {'phi_offset': 17.132087707519528, 'theta_offset': -31.953716278076172}
Left eye offsets: {'phi_offset': -4.765095924234248, 'theta_offset': -22.920942243820747}
Right eye 

In [8]:
def create_saccade_events_with_direction_segmentation_robust(
        eye_data_df,
        speed_threshold,  # angular speed threshold in degrees/frame
        directional_delta_threshold_deg=25,  # threshold for change in instantaneous angle (degrees)
        magnitude_calib=1,
        speed_profile=True,
        min_subsaccade_samples=2,
        min_net_disp=0.5  # minimal net angular displacement (in degrees) for a segment to be valid
):
    """
    Detects and segments saccade events in eye tracking data using angular speed and directional changes.
    This robust version avoids producing segments with near-zero net displacement by:
      1. Defining the saccade onset as the first frame where the angular speed exceeds the threshold.
      2. Segmenting the event based on sustained directional changes.
      3. Discarding segments whose overall net angular displacement (computed from k_phi and k_theta)
         is below a user-specified minimal value.

    Parameters:
      - eye_data_df (pd.DataFrame): DataFrame with columns including:
            'center_x', 'center_y', 'k_phi', 'k_theta', 'OE_timestamp', 'ms_axis', 'pupil_diameter'.
      - speed_threshold (float): Angular speed threshold (degrees/frame) for detection.
      - directional_delta_threshold_deg (float): Angular change threshold to determine segmentation boundaries.
      - magnitude_calib (float): Calibration factor (not applied to angular measures).
      - speed_profile (bool): Whether to record speed profiles.
      - min_subsaccade_samples (int): Minimum number of samples required for a valid segment.
      - min_net_disp (float): Minimal net angular displacement (in degrees) required for a segment to be kept.

    Returns:
      - df (pd.DataFrame): The input DataFrame with added computed columns.
      - saccade_events_df (pd.DataFrame): DataFrame listing detected and segmented saccade events, with metrics.
    """
    # Make a copy so as not to modify the original DataFrame.
    df = eye_data_df.copy()

    ### 1. Compute Frame-to-Frame Differences
    df["speed_x"] = df["center_x"].diff()
    df["speed_y"] = df["center_y"].diff()
    df["speed_r"] = np.sqrt(df["speed_x"] ** 2 + df["speed_y"] ** 2)

    # Angular differences (k_phi and k_theta are in degrees)
    df["angular_speed_phi"] = df["k_phi"].diff()
    df["angular_speed_theta"] = df["k_theta"].diff()
    df["angular_speed_r"] = np.sqrt(df["angular_speed_phi"] ** 2 + df["angular_speed_theta"] ** 2)

    ### 2. Saccade Detection Based on Angular Speed
    # Mark frames where the instantaneous angular speed exceeds the threshold.
    df["is_saccade_angle"] = df["angular_speed_r"] > speed_threshold

    # Identify transitions to detect onsets and offsets.
    saccade_on_off = df["is_saccade_angle"].astype(int) - df["is_saccade_angle"].shift(1, fill_value=0).astype(int)
    # Use the first frame above threshold as onset
    saccade_on_inds = np.where(saccade_on_off == 1)[0]
    saccade_off_inds = np.where(saccade_on_off == -1)[0]

    # Handle mismatches: if a saccade starts but does not end, drop the last onset.
    if len(saccade_on_inds) > len(saccade_off_inds):
        saccade_on_inds = saccade_on_inds[:-1]

    saccade_events = []

    ### 3. Process Each Detected Saccade for Segmentation
    for start_ind, end_ind in zip(saccade_on_inds, saccade_off_inds):
        saccade_df = df.iloc[start_ind:end_ind + 1].copy()
        if saccade_df.empty or len(saccade_df) < min_subsaccade_samples:
            continue

        # Compute instantaneous angles (for both pixel- and angular-based estimates)
        saccade_df["inst_angle_pixel"] = np.degrees(np.arctan2(saccade_df["speed_y"], saccade_df["speed_x"]))
        saccade_df["inst_angle_deg"] = np.degrees(
            np.arctan2(saccade_df["angular_speed_theta"], saccade_df["angular_speed_phi"]))

        # Define helper function for minimal angular difference (handling circularity)
        minimal_angle_diff_deg = lambda a, b: ((a - b + 180) % 360) - 180

        angles = saccade_df["inst_angle_deg"].values
        # Compute consecutive differences
        angle_diffs = np.array([minimal_angle_diff_deg(angles[i + 1], angles[i]) for i in range(len(angles) - 1)])

        # Identify candidate segmentation boundaries when the absolute change exceeds threshold.
        candidate_boundaries = np.where(np.abs(angle_diffs) > directional_delta_threshold_deg)[0].tolist()

        # Always include the first and last frame of the saccade.
        boundaries = [0] + candidate_boundaries + [len(saccade_df) - 1]

        # Process each segment defined by these boundaries.
        for i in range(len(boundaries) - 1):
            seg_start = boundaries[i]
            seg_end = boundaries[i + 1]
            subsaccade = saccade_df.iloc[seg_start: seg_end + 1]
            if len(subsaccade) < min_subsaccade_samples:
                continue

            # Compute net angular displacement using the angular positions (k_phi and k_theta)
            initial_pos_angle = subsaccade.iloc[0][["k_phi", "k_theta"]]
            final_pos_angle = subsaccade.iloc[-1][["k_phi", "k_theta"]]
            net_disp = np.sqrt((final_pos_angle["k_phi"] - initial_pos_angle["k_phi"]) ** 2 +
                               (final_pos_angle["k_theta"] - initial_pos_angle["k_theta"]) ** 2)

            # Only record segments whose net displacement is above min_net_disp.
            if net_disp < min_net_disp:
                continue

            # Timing and indices
            sub_start_timestamp = subsaccade["OE_timestamp"].iloc[0]
            sub_end_timestamp = subsaccade["OE_timestamp"].iloc[-1]
            sub_start_ms = subsaccade["ms_axis"].iloc[0]
            sub_end_ms = subsaccade["ms_axis"].iloc[-1]
            sub_length = subsaccade.index[-1] - subsaccade.index[0]

            # Pixel-based metrics
            magnitude_raw_pixel = subsaccade["speed_r"].sum()
            magnitude_pixel = magnitude_raw_pixel * magnitude_calib

            # Angular-based metric: sum of instantaneous angular speeds
            magnitude_raw_angular = subsaccade["angular_speed_r"].sum()

            # Overall angular-based angle (from start to end)
            overall_angle_deg = (np.degrees(np.arctan2(
                final_pos_angle["k_theta"] - initial_pos_angle["k_theta"],
                final_pos_angle["k_phi"] - initial_pos_angle["k_phi"]
            )) % 360)

            # (Optional) Capture speed profiles and other details
            speed_profile_pixel = subsaccade["speed_r"].values if speed_profile else None
            speed_profile_pixel_calib = (speed_profile_pixel * magnitude_calib) if speed_profile else None
            speed_profile_angular = subsaccade["angular_speed_r"].values if speed_profile else None
            diameter_profile = subsaccade["pupil_diameter"].values

            saccade_events.append({
                "saccade_start_ind": subsaccade.index[0],
                "saccade_end_ind": subsaccade.index[-1],
                "saccade_start_timestamp": sub_start_timestamp,
                "saccade_end_timestamp": sub_end_timestamp,
                "saccade_on_ms": sub_start_ms,
                "saccade_off_ms": sub_end_ms,
                "length": sub_length,
                "magnitude_raw_pixel": magnitude_raw_pixel,
                "magnitude_pixel": magnitude_pixel,
                "magnitude_raw_angular": magnitude_raw_angular,
                "overall_angle_deg": overall_angle_deg,
                "net_angular_disp": net_disp,
                "speed_profile_pixel": speed_profile_pixel,
                "speed_profile_pixel_calib": speed_profile_pixel_calib,
                "speed_profile_angular": speed_profile_angular,
                "diameter_profile": diameter_profile,
                "theta_init_pos": initial_pos_angle["k_theta"],
                "theta_end_pos": final_pos_angle["k_theta"],
                "phi_init_pos": initial_pos_angle["k_phi"],
                "phi_end_pos": final_pos_angle["k_phi"]
            })

    # Convert the list to a DataFrame.
    saccade_events_df = pd.DataFrame(saccade_events)

    # Optionally remove intermediate column
    df.drop(["is_saccade_angle"], axis=1, inplace=True)

    # Calculate delta columns for convenience.
    if not saccade_events_df.empty:
        saccade_events_df['delta_theta'] = saccade_events_df['theta_end_pos'] - saccade_events_df['theta_init_pos']
        saccade_events_df['delta_phi'] = saccade_events_df['phi_end_pos'] - saccade_events_df['phi_init_pos']

    return df, saccade_events_df

In [9]:

for block in block_collection:
    print(block)
    block.left_eye_data, block.l_saccade_df = create_saccade_events_with_direction_segmentation_robust(
        block.left_eye_data,
        0.8,  # angular speed threshold in degrees/frame
        directional_delta_threshold_deg=90,  # threshold for change in instantaneous angle (degrees)
        magnitude_calib=1,
        speed_profile=True,
        min_subsaccade_samples=2)
    block.right_eye_data, block.r_saccade_df = create_saccade_events_with_direction_segmentation_robust(
        block.right_eye_data,
        0.8,  # angular speed threshold in degrees/frame
        directional_delta_threshold_deg=90,  # threshold for change in instantaneous angle (degrees)
        magnitude_calib=1,
        speed_profile=True,
        min_subsaccade_samples=2)


PV_106, block 008, on PV106_ET_d3t12025-08-06_11-52-19
PV_106, block 009, on PV106_ET_d3t2_2025-08-06_12-09-43
PV_106, block 010, on PV106_ET_d3t3_2025-08-06_12-26-43
PV_106, block 011, on PV106_ET_d3t4_2025-08-06_12-44-30
PV_106, block 012, on PV106_ET_d3t5_2025-08-06_13-21-30
PV_143, block 001, on PV143_ET_d1t1_2025-08-11_13-29-08
PV_143, block 002, on PV143_ET_d1t2_2025-08-11_13-50-11
PV_143, block 003, on PV143_ET_d1t3_2025-08-11_14-20-35
PV_143, block 004, on PV143_ET_d1t4_2025-08-11_14-58-28
PV_62, block 024, on 2023-04-27_11-22-56
PV_62, block 026, on 2023-04-27_12-21-41
PV_62, block 038, on 2023-05-01_13-57-45
PV_126, block 007, on PV126_Trial16_wake3_2024-07-18_12-49-12
PV_126, block 008, on PV126_Trial16_wake4_2024-07-18_13-24-41
PV_126, block 009, on PV126_Trial18_wake5_2024-07-18_14-39-15
PV_126, block 010, on PV126_Trial19_wake6_2024-07-18_15-24-57
PV_126, block 011, on PV126_Trial115_eyeTracking_w7
PV_126, block 012, on PV126_Trial116_eyeTracking_h8
PV_57, block 007, on p

In [10]:
### over here, I get the lizard movements binary from mark's analysis
import os
def block_get_lizard_movement(block):
    # collect accelerometer data
    # path definition
    p = block.oe_path / 'analysis'
    analysis_list = os.listdir(p)
    correct_analysis = [i for i in analysis_list if block.animal_call in i][0]
    p = p / str(correct_analysis)
    mat_path = p / 'lizMov.mat'
    print(f'path to mat file is {mat_path}')
    # read mat file
    try:
        mat_data = h5py.File(str(mat_path), 'r')
        mat_dict = {'t_mov_ms': mat_data['t_mov_ms'][:],
                    'movAll': mat_data['movAll'][:]}

        acc_df = pd.DataFrame(data=np.array([mat_dict['t_mov_ms'][:, 0], mat_dict['movAll'][:, 0]]).T,
                              columns=['t_mov_ms', 'movAll'])
        mat_data.close()
        block.liz_mov_df = acc_df
        print(f'liz_mov_df created for {block}')
    except FileNotFoundError:
        print('mat file does not exist - run the matlab getLizMovement function')

    return


# this cell now labels the saccades as with/without head movements
def get_head_mov_col(df, mov_times):
    head_mov_bool = np.zeros(len(df), dtype=bool)  # Initialize array of False

    for i, saccade in enumerate(df.itertuples()):
        saccade_start = saccade.saccade_on_ms
        saccade_end = saccade.saccade_off_ms

        overlapping_mov_times = mov_times[np.logical_and(mov_times >= saccade_start, mov_times <= saccade_end)]

        if overlapping_mov_times.size > 0:
            head_mov_bool[i] = True

    df['head_movement'] = head_mov_bool
    return df

def label_saccade_movements(block):
    mov_times = block.liz_mov_df.t_mov_ms.values
    block.l_saccade_df = get_head_mov_col(block.l_saccade_df,mov_times=mov_times)
    block.r_saccade_df = get_head_mov_col(block.r_saccade_df,mov_times=mov_times)

# Create a list to store blocks where movement data exists
block_collection_w_mov = []

for block in block_collection:
    try:
        block_get_lizard_movement(block)  # Try loading movement data
        label_saccade_movements(block)    # Try labeling saccades

        # If both steps succeed, add block to the valid collection
        block_collection_w_mov.append(block)

    except (FileNotFoundError, OSError) as e:
        print(f"Skipping block {block}: {str(e)}")  # Notify which block failed


path to mat file is Z:\Nimrod\experiments\PV_106\2025_08_06\block_008\oe_files\PV106_ET_d3t12025-08-06_11-52-19\Record Node 102\analysis\recNames=Block008,Animal=PV_106\lizMov.mat
liz_mov_df created for PV_106, block 008, on PV106_ET_d3t12025-08-06_11-52-19
path to mat file is Z:\Nimrod\experiments\PV_106\2025_08_06\block_009\oe_files\PV106_ET_d3t2_2025-08-06_12-09-43\Record Node 102\analysis\recNames=Block009,Animal=PV_106\lizMov.mat
liz_mov_df created for PV_106, block 009, on PV106_ET_d3t2_2025-08-06_12-09-43
path to mat file is Z:\Nimrod\experiments\PV_106\2025_08_06\block_010\oe_files\PV106_ET_d3t3_2025-08-06_12-26-43\Record Node 102\analysis\recNames=Block010,Animal=PV_106\lizMov.mat
liz_mov_df created for PV_106, block 010, on PV106_ET_d3t3_2025-08-06_12-26-43
path to mat file is Z:\Nimrod\experiments\PV_106\2025_08_06\block_011\oe_files\PV106_ET_d3t4_2025-08-06_12-44-30\Record Node 102\analysis\recNames=Block011,Animal=PV_106\lizMov.mat
liz_mov_df created for PV_106, block 011,

In [11]:
# add annotations for a joint dataframe:
for block in block_collection:
    block.r_saccade_df['eye'] = 'R'
    block.r_saccade_df['block'] = block.block_num
    block.r_saccade_df['animal'] = block.animal_call
    block.l_saccade_df['eye'] = 'L'
    block.l_saccade_df['block'] = block.block_num
    block.l_saccade_df['animal'] = block.animal_call
    block.all_saccade_df = pd.concat([block.l_saccade_df,block.r_saccade_df])

In [12]:
saccade_df_list = []
for block in block_collection:
    saccade_df_list.append(block.all_saccade_df)
saccade_collection = pd.concat(saccade_df_list)

In [13]:
def get_closest_diff_ind(timestamp, timeseries):
    """
    This function extracts a frame from a series so that it is as close as possible to a given timestamp
    :param timestamp: The time to match a frame to
    :param timeseries: The time frames series to look at for a match
    :param report_acc: if set to 1, will report the accuracy of the match
    :return: index_of_lowest_diff , accuracy of match (if requested)
    """
    array = np.abs(timeseries - timestamp)
    index_of_lowest_diff = np.argmin(array)
    lowest_diff_val = timeseries[index_of_lowest_diff]
    return index_of_lowest_diff, lowest_diff_val

def find_synced_saccades(df, diff_threshold=680):
    synced_saccades = []
    non_synced_saccades = []
    l_df = df.query('eye == "L"')
    r_df = df.query('eye == "R"')
    for i, row in tqdm.tqdm(l_df.iterrows()):
        l_timestamp = row['saccade_start_timestamp']
        ind_min_diff, r_timestamp = get_closest_diff_ind(l_timestamp, r_df['saccade_start_timestamp'].values)
        #print(i,ind_lowest_diff)
        time_diff = np.abs(l_timestamp - r_timestamp)
        if time_diff < diff_threshold:
            synced_saccades.append((row, r_df.iloc[ind_min_diff]))  # Collect synchronized rows
        else:
            non_synced_saccades.append(row)  # Collect non-synchronized rows
    # Create DataFrame with multi-index
    multi_index = pd.MultiIndex.from_tuples([(i, 'L') for i in range(len(synced_saccades))] + [(i, 'R') for i in range(len(synced_saccades))], names=['Main', 'Sub'])
    synced_df = pd.DataFrame(index=multi_index, columns=df.columns)
    # Populate DataFrame
    for idx, (l_row, r_row) in enumerate(synced_saccades):
        synced_df.loc[(idx, 'L')] = l_row
        synced_df.loc[(idx, 'R')] = r_row
    r_non_synced_leftovers = r_df[~r_df['saccade_start_timestamp'].isin(synced_df.query('eye == "R"')['saccade_start_timestamp'].values)]
    print(len(r_non_synced_leftovers),len(r_df))
    # Create DataFrame for non-synced saccades
    non_synced_df = pd.DataFrame(non_synced_saccades, columns=df.columns)
    non_synced_df = pd.concat([non_synced_df,r_non_synced_leftovers])

    return synced_df, non_synced_df

synced_df_list = []
non_synced_df_list = []
for saccade_df in saccade_df_list:
    # Find synced saccades:
    synced_df, non_synced_df = find_synced_saccades(saccade_df.dropna(), diff_threshold=680)
    if len(non_synced_df.dropna()) + len(synced_df.dropna()) == len(saccade_df.dropna()):
        print('got them all')

    synced_df_list.append(synced_df)
    non_synced_df_list.append(non_synced_df)


924it [00:00, 8062.92it/s]
395it [00:00, 7064.34it/s]
472it [00:00, 9466.14it/s]

105 517
102 336



1273it [00:00, 7883.73it/s]

103 320



724it [00:00, 4969.70it/s]

...

461 1087



786it [00:00, 5741.14it/s]
635it [00:00, 6334.10it/s]

487 888
286 763


999it [00:00, 4866.89it/s]


573 1245


In [14]:
def combine_synced_dataframes(dataframes):
    combined_dfs = []
    start_index = 0
    for df in dataframes:
        num_rows = len(df) // 2  # Assuming each dataframe contains pairs of rows
        main_index = pd.MultiIndex.from_tuples([(i + start_index, 'L') for i in range(num_rows)] + [(i + start_index, 'R') for i in range(num_rows)], names=['Main', 'Sub'])
        df.index = main_index
        combined_dfs.append(df)
        start_index += num_rows
    combined_df = pd.concat(combined_dfs)
    combined_df.reset_index(inplace=True)
    return combined_df

synced_saccade_collection = combine_synced_dataframes(synced_df_list)
non_synced_saccade_collection = pd.concat(non_synced_df_list)



In [19]:
all_saccade_collection = pd.concat([synced_saccade_collection,non_synced_saccade_collection])

In [20]:
#clean monocular saccade df from false detections (actually not monocular):
import numpy as np
import pandas as pd

# ------------------------------------------------------------
# Helper: find left/right per-frame eye trace dfs in a block
# ------------------------------------------------------------
def _get_eye_df_from_block(block, candidates):
    for attr in candidates:
        if hasattr(block, attr):
            df = getattr(block, attr)
            if isinstance(df, pd.DataFrame) and len(df) > 0:
                return df
    return None


def build_eye_trace_lookup_from_block_collection(
    block_collection,
    animal_attr="animal_call",
    block_attr="block_num",
    left_df_attrs=None,
    right_df_attrs=None,
):
    """
    Returns:
      eye_trace_lookup[(animal, block, 'L'/'R')] = per-frame eye dataframe

    Notes:
    - You may need to adjust left_df_attrs/right_df_attrs to match your BlockSync object fields.
    """
    if left_df_attrs is None:
        left_df_attrs = ["left_eye_data","l_eye_df", "left_eye_df", "l_df", "left_df", "l_eye_data_df", "left_eye_data_df"]
    if right_df_attrs is None:
        right_df_attrs = ["right_eye_data","r_eye_df", "right_eye_df", "r_df", "right_df", "r_eye_data_df", "right_eye_data_df"]

    lookup = {}
    for blk in block_collection:
        animal = getattr(blk, animal_attr, "unknown")
        bnum = getattr(blk, block_attr, "unknown")

        ldf = _get_eye_df_from_block(blk, left_df_attrs)
        rdf = _get_eye_df_from_block(blk, right_df_attrs)

        if ldf is not None:
            lookup[(animal, bnum, "L")] = ldf
        if rdf is not None:
            lookup[(animal, bnum, "R")] = rdf

    return lookup


# ------------------------------------------------------------
# Core: verify monocular by checking other-eye speed near onset
# ------------------------------------------------------------
def annotate_monocular_by_other_eye_speed(
    non_synced_df,
    eye_trace_lookup,
    t0_col="saccade_on_ms",
    eye_col="eye",
    animal_col="animal",
    block_col="block",
    ms_col="ms_axis",
    speed_col="angular_speed_r",
    window_ms=80,
    speed_thresh=5.0,
):
    """
    Adds:
      - other_eye_peak_speed
      - verified_monocular (True/False/NaN)
        True  = other-eye peak speed < speed_thresh in +/- window_ms
        False = other-eye peak speed >= speed_thresh
        NaN   = missing trace/cols/window

    Returns: annotated dataframe
    """
    d = non_synced_df.copy()

    # Support alternative eye labeling if needed (e.g., 'Sub' == 'L'/'R')
    if eye_col not in d.columns and "Sub" in d.columns:
        d[eye_col] = d["Sub"]

    peaks = []
    verified = []

    for _, r in d.iterrows():
        try:
            animal = r[animal_col]
            block = r[block_col]
            eye = r[eye_col]
            t0 = float(r[t0_col])
        except Exception:
            peaks.append(np.nan)
            verified.append(np.nan)
            continue

        if not np.isfinite(t0) or eye not in ("L", "R"):
            peaks.append(np.nan)
            verified.append(np.nan)
            continue

        other_eye = "R" if eye == "L" else "L"
        key = (animal, block, other_eye)

        if key not in eye_trace_lookup:
            peaks.append(np.nan)
            verified.append(np.nan)
            continue

        tr = eye_trace_lookup[key]
        if (ms_col not in tr.columns) or (speed_col not in tr.columns):
            peaks.append(np.nan)
            verified.append(np.nan)
            continue

        ms = tr[ms_col].astype(float).to_numpy()
        sp = tr[speed_col].astype(float).to_numpy()

        m = (ms >= (t0 - window_ms)) & (ms <= (t0 + window_ms)) & np.isfinite(sp)
        if not np.any(m):
            peaks.append(np.nan)
            verified.append(np.nan)
            continue

        peak = float(np.nanmax(sp[m]))
        peaks.append(peak)
        verified.append(bool(peak < float(speed_thresh)))

    d["other_eye_peak_speed"] = np.asarray(peaks, dtype=float)
    d["verified_monocular"] = verified
    return d


def create_verified_monocular_saccade_collection(
    non_synced_saccade_collection,
    block_collection,
    t0_col="saccade_on_ms",
    ms_col="ms_axis",
    speed_col="angular_speed_r",
    window_ms=80,
    speed_thresh=5.0,
    keep_unverifiable=False,
    verbose=True,
):
    """
    Pipeline entry-point:
      Input: non_synced_saccade_collection (your monocular candidates)
      Output:
        - verified_monocular_saccade_collection (subset)
        - annotated_non_synced_df (with other_eye_peak_speed + verified_monocular column)
        - diagnostics dict
    """
    eye_trace_lookup = build_eye_trace_lookup_from_block_collection(block_collection)

    annotated = annotate_monocular_by_other_eye_speed(
        non_synced_df=non_synced_saccade_collection,
        eye_trace_lookup=eye_trace_lookup,
        t0_col=t0_col,
        ms_col=ms_col,
        speed_col=speed_col,
        window_ms=window_ms,
        speed_thresh=speed_thresh,
    )

    vc = annotated["verified_monocular"].value_counts(dropna=False)
    n_true = int(vc.get(True, 0))
    n_false = int(vc.get(False, 0))
    n_nan = int(vc.get(np.nan, 0))
    n_total = len(annotated)

    if keep_unverifiable:
        # treat NaN as "not verified but keep"
        verified_df = annotated.loc[(annotated["verified_monocular"] == True) | (annotated["verified_monocular"].isna())].copy()
    else:
        verified_df = annotated.loc[annotated["verified_monocular"] == True].copy()

    diagnostics = {
        "lookup_entries": len(eye_trace_lookup),
        "n_total_non_synced": n_total,
        "n_verified_true": n_true,
        "n_verified_false": n_false,
        "n_verified_nan": n_nan,
        "verified_fraction_of_classifiable": (n_true / float(n_true + n_false)) if (n_true + n_false) > 0 else np.nan,
        "nan_fraction": (n_nan / float(n_total)) if n_total > 0 else np.nan,
        "speed_col": speed_col,
        "speed_thresh": speed_thresh,
        "window_ms": window_ms,
    }

    if verbose:
        print("Lookup entries:", diagnostics["lookup_entries"])
        print("verified_monocular counts:", vc.to_dict())
        print("Verified fraction (of classifiable True/False):", diagnostics["verified_fraction_of_classifiable"])

    return verified_df, annotated, diagnostics

verified_monocular_saccade_collection, non_synced_annotated, mono_diag = create_verified_monocular_saccade_collection(
    non_synced_saccade_collection=non_synced_saccade_collection,
    block_collection=block_collection,          # IMPORTANT: use the full/original one
    speed_col="angular_speed_r",                # or "speed_r" if that's what your eye dfs have
    ms_col="ms_axis",
    t0_col="saccade_on_ms",
    window_ms=80,
    speed_thresh=0.8,                   # use the threshold you just tuned
    keep_unverifiable=False,                    # strict
    verbose=True,
)


Lookup entries: 46
verified_monocular counts: {False: 7292, True: 5321, nan: 1679}
Verified fraction (of classifiable True/False): 0.42186632839134225


In [21]:
import numpy as np
import pandas as pd
from typing import Tuple

def compute_event_stats_from_synced_tables(
    synced_saccade_collection: pd.DataFrame,
    non_synced_saccade_collection: pd.DataFrame,
    head_col: str = "head_movement",
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Uses your existing verified annotation:
      - synced_saccade_collection: paired L/R saccades, with columns ['Main','Sub', ...] after reset_index()
      - non_synced_saccade_collection: unpaired single-eye saccades

    Returns:
      per_animal_df, overall_df
    """

    # -----------------------
    # 1) Build event-level df
    # -----------------------
    events = []

    # Binocular events: group by (animal, block, Main) in synced collection
    if len(synced_saccade_collection) > 0:
        required = set(["animal", "block", "Main", head_col])
        missing = required - set(synced_saccade_collection.columns)
        if missing:
            raise ValueError("synced_saccade_collection missing columns: %s" % sorted(list(missing)))

        gcols = ["animal", "block", "Main"]
        for (animal, block, main), g in synced_saccade_collection.groupby(gcols, dropna=False):
            # event is binocular by definition
            # head movement at event-level: OR across the two eyes (robust)
            head_m = bool(np.any(g[head_col].astype(bool).to_numpy()))
            events.append(dict(
                animal=animal,
                block=block,
                event_type="binocular",
                head_movement=head_m,
            ))

    # Monocular events: each row is one event
    if len(non_synced_saccade_collection) > 0:
        required = set(["animal", "block", head_col])
        missing = required - set(non_synced_saccade_collection.columns)
        if missing:
            raise ValueError("non_synced_saccade_collection missing columns: %s" % sorted(list(missing)))

        for _, r in non_synced_saccade_collection.iterrows():
            events.append(dict(
                animal=r["animal"],
                block=r["block"],
                event_type="monocular",
                head_movement=bool(r[head_col]),
            ))

    events = pd.DataFrame(events)
    if events.empty:
        raise RuntimeError("No events created. Check that synced/non-synced tables are non-empty.")

    # -----------------------
    # 2) Per-animal summary
    # -----------------------
    out = []
    for animal, aev in events.groupby("animal"):
        total = len(aev)
        still = aev.loc[~aev["head_movement"].astype(bool)]
        moving = aev.loc[aev["head_movement"].astype(bool)]

        def pct_monocular(df):
            if len(df) == 0:
                return np.nan
            return 100.0 * np.mean(df["event_type"].to_numpy() == "monocular")

        out.append(dict(
            animal=animal,
            n_events=total,
            pct_events_head_still=100.0 * len(still) / total if total else np.nan,
            pct_events_head_moving=100.0 * len(moving) / total if total else np.nan,
            pct_monocular_when_head_still=pct_monocular(still),
            pct_monocular_when_head_moving=pct_monocular(moving),
            n_head_still=len(still),
            n_head_moving=len(moving),
        ))

    per_animal = pd.DataFrame(out).sort_values("animal").reset_index(drop=True)

    # -----------------------
    # 3) Overall summary
    # -----------------------
    def mean_sem(x):
        x = np.asarray(x, dtype=float)
        x = x[np.isfinite(x)]
        if x.size == 0:
            return np.nan, np.nan
        mean = float(np.mean(x))
        sem = float(np.std(x, ddof=1) / np.sqrt(x.size)) if x.size > 1 else np.nan
        return mean, sem

    pooled_pct_head_still = 100.0 * np.mean(~events["head_movement"].astype(bool))
    pooled_pct_mono_still = (
        100.0 * np.mean(events.loc[~events["head_movement"].astype(bool), "event_type"].to_numpy() == "monocular")
        if np.any(~events["head_movement"].astype(bool)) else np.nan
    )
    pooled_pct_mono_moving = (
        100.0 * np.mean(events.loc[events["head_movement"].astype(bool), "event_type"].to_numpy() == "monocular")
        if np.any(events["head_movement"].astype(bool)) else np.nan
    )

    m1, s1 = mean_sem(per_animal["pct_events_head_still"])
    m2, s2 = mean_sem(per_animal["pct_monocular_when_head_still"])
    m3, s3 = mean_sem(per_animal["pct_monocular_when_head_moving"])

    overall = pd.DataFrame([
        dict(metric="pct_events_head_still",
             mean_across_animals=m1, sem_across_animals=s1,
             pooled_value=pooled_pct_head_still, pooled_n_events=len(events)),
        dict(metric="pct_monocular_when_head_still",
             mean_across_animals=m2, sem_across_animals=s2,
             pooled_value=pooled_pct_mono_still, pooled_n_events=len(events)),
        dict(metric="pct_monocular_when_head_moving",
             mean_across_animals=m3, sem_across_animals=s3,
             pooled_value=pooled_pct_mono_moving, pooled_n_events=len(events)),
    ])

    return per_animal, overall
compute_event_stats_from_synced_tables(synced_saccade_collection,verified_monocular_saccade_collection)

(   animal  n_events  pct_events_head_still  pct_events_head_moving  \
 0  PV_106      3398              40.435550               59.564450   
 1  PV_126      3456              39.409722               60.590278   
 2  PV_143      1240              29.677419               70.322581   
 3   PV_57      5254               0.000000              100.000000   
 4   PV_62      2265              56.600442               43.399558   
 
    pct_monocular_when_head_still  pct_monocular_when_head_moving  \
 0                      70.524017                       13.982213   
 1                      27.973568                       15.759312   
 2                      45.923913                       12.041284   
 3                            NaN                       41.720594   
 4                      51.170047                       24.008138   
 
    n_head_still  n_head_moving  
 0          1374           2024  
 1          1362           2094  
 2           368            872  
 3             0    

In [22]:
# verify the numerousity of monocular events here:
import numpy as np
import pandas as pd

def relabel_pairs_by_window_overlap(
    synced_saccade_collection: pd.DataFrame,
    non_synced_saccade_collection: pd.DataFrame,
    all_saccade_collection: pd.DataFrame,
    on_col: str = "saccade_on_ms",
    off_col: str = "saccade_off_ms",
    eye_col: str = "eye",
    animal_col: str = "animal",
    block_col: str = "block",
    max_onset_dt_ms: float = 40.0,
    min_overlap_ms: float = 0.0,
):
    """
    Re-check each 'non_synced' event: does the other eye have a saccade whose window overlaps?
    If yes -> reclassify as binocular (was "missed pair").
    Requires all_saccade_collection to include BOTH eyes' saccade windows (L+R) with on/off times.
    """

    # Start by treating current synced as binocular events; non_synced as monocular candidates
    non = non_synced_saccade_collection.copy()

    # Make sure we know which eye each non-synced event belongs to
    if eye_col not in non.columns:
        raise ValueError("non_synced_saccade_collection must include an 'eye' column (L/R).")

    # We'll look up candidate matches in the opposite eye within each animal+block
    all_df = all_saccade_collection.copy()

    # Basic guards
    for c in [animal_col, block_col, eye_col, on_col, off_col]:
        if c not in all_df.columns:
            raise ValueError(f"all_saccade_collection missing required column: {c}")

    # Index all events by animal+block for fast filtering
    grouped = dict(tuple(all_df.groupby([animal_col, block_col], dropna=False)))

    reclassified = np.zeros(len(non), dtype=bool)

    for i, r in non.iterrows():
        key = (r[animal_col], r[block_col])
        if key not in grouped:
            continue

        eye = r[eye_col]
        other_eye = "R" if eye == "L" else "L"
        s0 = float(r[on_col])
        s1 = float(r[off_col])
        if not (np.isfinite(s0) and np.isfinite(s1)):
            continue

        cand = grouped[key]
        cand = cand[cand[eye_col] == other_eye]

        if cand.empty:
            continue

        # onset proximity constraint (optional but helps avoid nonsense matches)
        cand_on = cand[on_col].astype(float).to_numpy()
        onset_dt = np.abs(cand_on - s0)
        cand = cand.loc[onset_dt <= max_onset_dt_ms]
        if cand.empty:
            continue

        # overlap condition
        c0 = cand[on_col].astype(float).to_numpy()
        c1 = cand[off_col].astype(float).to_numpy()
        overlap = np.minimum(s1, c1) - np.maximum(s0, c0)  # positive => overlap in ms

        if np.any(overlap >= min_overlap_ms):
            reclassified[i] = True

    # Build adjusted event tables
    non["reclassified_as_binocular"] = reclassified
    adj_non_synced = non.loc[~non["reclassified_as_binocular"]].copy()

    # Return updated non-synced + a count
    return adj_non_synced, int(np.sum(reclassified))

adj_non, n_repaired = relabel_pairs_by_window_overlap(
    synced_saccade_collection,
    non_synced_saccade_collection,
    all_saccade_collection,
    max_onset_dt_ms=40,     # start with 20–60 ms
    min_overlap_ms=0,       # or try 5–10 ms
)
print("Repaired (formerly 'monocular' but window-overlap matched):", n_repaired)


Repaired (formerly 'monocular' but window-overlap matched): 795


In [23]:
def confirm_monocular_by_other_eye_speed(
    non_synced_df: pd.DataFrame,
    eye_trace_lookup: dict,
    # eye_trace_lookup[(animal, block, eye)] -> dataframe with ms_axis and speed column
    t0_col="saccade_on_ms",
    eye_col="eye",
    animal_col="animal",
    block_col="block",
    ms_col="ms_axis",
    speed_col="angular_speed_r",
    window_ms=80,
    speed_thresh=150,  # set based on your speed units/distribution
):
    """
    Labels each non-synced event as 'confirmed_monocular' if the other eye has NO speed peak
    above speed_thresh in +/- window_ms around event onset.
    """
    d = non_synced_df.copy()
    confirmed = []

    for _, r in d.iterrows():
        animal, block, eye = r[animal_col], r[block_col], r[eye_col]
        other_eye = "R" if eye == "L" else "L"
        t0 = float(r[t0_col])

        key = (animal, block, other_eye)
        if key not in eye_trace_lookup:
            confirmed.append(np.nan)
            continue

        tr = eye_trace_lookup[key]
        if ms_col not in tr.columns or speed_col not in tr.columns:
            confirmed.append(np.nan)
            continue

        ms = tr[ms_col].astype(float).to_numpy()
        sp = tr[speed_col].astype(float).to_numpy()

        m = (ms >= t0 - window_ms) & (ms <= t0 + window_ms) & np.isfinite(sp)
        if not np.any(m):
            confirmed.append(np.nan)
            continue

        peak = np.nanmax(sp[m])
        confirmed.append(bool(peak < speed_thresh))

    d["confirmed_monocular_by_speed"] = confirmed
    return d

import numpy as np
import pandas as pd

# --- paste/keep your function definition somewhere above this cell ---
# def confirm_monocular_by_other_eye_speed(...): ...


def build_eye_trace_lookup_from_block_collection(
    block_collection,
    eye_df_attr_left="left_eye_data",
    eye_df_attr_right="right_eye_data",
    animal_attr="animal_call",
    block_attr="block_num",
):
    """
    Returns:
      eye_trace_lookup[(animal, block, 'L'/'R')] = per-frame dataframe
    """
    eye_trace_lookup = {}

    for blk in block_collection:
        animal = getattr(blk, animal_attr, "unknown")
        bnum = getattr(blk, block_attr, "unknown")

        ldf = getattr(blk, eye_df_attr_left, None)
        rdf = getattr(blk, eye_df_attr_right, None)

        if isinstance(ldf, pd.DataFrame) and len(ldf) > 0:
            eye_trace_lookup[(animal, bnum, "L")] = ldf

        if isinstance(rdf, pd.DataFrame) and len(rdf) > 0:
            eye_trace_lookup[(animal, bnum, "R")] = rdf

    return eye_trace_lookup


# ------------------------
# 1) Build lookup table
# ------------------------
eye_trace_lookup = build_eye_trace_lookup_from_block_collection(block_collection)

print("Lookup entries:", len(eye_trace_lookup))
# Example keys: next(iter(eye_trace_lookup.keys()))


# ------------------------
# 2) Run confirmation test
# ------------------------
# IMPORTANT:
# - t0_col should match what you want to anchor on (onset is usually best): 'saccade_on_ms'
# - ms_col must exist in eye dfs: usually 'ms_axis'
# - speed_col must exist: try 'angular_speed_r' first, else maybe 'speed_r'
# - speed_thresh must be in the same units as speed_col

non_confirmed = confirm_monocular_by_other_eye_speed(
    non_synced_df=non_synced_saccade_collection,
    eye_trace_lookup=eye_trace_lookup,
    t0_col="saccade_on_ms",
    eye_col="eye",
    animal_col="animal",
    block_col="block",
    ms_col="ms_axis",
    speed_col="angular_speed_r",
    window_ms=80,          # start with ±80 ms; also try 50/100
    speed_thresh=0.8,
)

# ------------------------
# 3) Summarize results
# ------------------------
# confirmed_monocular_by_speed == True  -> other eye stayed below threshold (confirmed monocular)
# confirmed_monocular_by_speed == False -> other eye had a speed peak (likely "missed binocular")
# confirmed_monocular_by_speed == NaN   -> missing trace / columns / window

print(non_confirmed["confirmed_monocular_by_speed"].value_counts(dropna=False))

# Optional: confirm rates by head condition (if head_movement exists in non_synced df)
if "head_movement" in non_confirmed.columns:
    tmp = non_confirmed.copy()
    tmp["head_state"] = np.where(tmp["head_movement"].astype(bool), "moving", "still")
    print(
        tmp.groupby("head_state")["confirmed_monocular_by_speed"]
           .value_counts(normalize=True, dropna=False)
    )


# ------------------------
# 4) (Recommended) Tune speed_thresh empirically
# ------------------------
# Look at the distribution of the OTHER-eye peak speeds around non-synced events
# by temporarily modifying the function to store peak, or do a quick estimate here:

def compute_other_eye_peak_speeds(
    non_synced_df, eye_trace_lookup,
    t0_col="saccade_on_ms", eye_col="eye", animal_col="animal", block_col="block",
    ms_col="ms_axis", speed_col="angular_speed_r",
    window_ms=80
):
    peaks = []
    for _, r in non_synced_df.iterrows():
        animal, block, eye = r[animal_col], r[block_col], r[eye_col]
        other_eye = "R" if eye == "L" else "L"
        t0 = float(r[t0_col])
        key = (animal, block, other_eye)
        if key not in eye_trace_lookup:
            peaks.append(np.nan)
            continue
        tr = eye_trace_lookup[key]
        if ms_col not in tr.columns or speed_col not in tr.columns:
            peaks.append(np.nan)
            continue
        ms = tr[ms_col].astype(float).to_numpy()
        sp = tr[speed_col].astype(float).to_numpy()
        m = (ms >= t0 - window_ms) & (ms <= t0 + window_ms) & np.isfinite(sp)
        if not np.any(m):
            peaks.append(np.nan)
        else:
            peaks.append(float(np.nanmax(sp[m])))
    return np.asarray(peaks, dtype=float)

peaks = compute_other_eye_peak_speeds(
    non_synced_saccade_collection, eye_trace_lookup,
    speed_col="angular_speed_r", window_ms=80
)

print("Other-eye peak speed percentiles (non-synced events):",
      np.nanpercentile(peaks, [50, 75, 90, 95, 99]))


Lookup entries: 46
False    7292
True     5321
NaN      1679
Name: confirmed_monocular_by_speed, dtype: int64
head_state  confirmed_monocular_by_speed
moving      False                           0.575376
            True                            0.338280
            NaN                             0.086344
still       True                            0.435697
            False                           0.388822
            NaN                             0.175481
Name: confirmed_monocular_by_speed, dtype: float64
Other-eye peak speed percentiles (non-synced events): [ 1.06764261  2.8285699   5.01649533  6.59673655 11.82123613]


In [24]:
print(synced_saccade_collection.groupby(["animal","block","Main"]).size().value_counts())


2    10292
dtype: int64


In [25]:
from pathlib import Path
from datetime import datetime
from typing import Optional, Union, List, Dict, Tuple

import numpy as np
import pandas as pd
import cv2

# =========================
# Sterile annotation settings
# =========================
CURATION_DIRNAME = "curation_monocular_video"     # under each block.analysis_path
CURATION_CSV_NAME = "monocular_video_curation.csv"

# tri-state: {True, False, None}
INCLUDE_COL = "include_in_monocular_video"
TS_COL = "annotation_timestamp"


def _first_existing(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None


def _require_col(df, candidates, what):
    c = _first_existing(df, candidates)
    if c is None:
        raise KeyError(
            "Missing {}. Tried: {}. Existing columns (first 40): {}".format(
                what, candidates, list(df.columns)[:40]
            )
        )
    return c


def _coerce_tristate_bool(v):
    if v is None:
        return None
    try:
        # np.nan handling
        if isinstance(v, float) and np.isnan(v):
            return None
    except Exception:
        pass

    if isinstance(v, (bool, np.bool_)):
        return bool(v)

    s = str(v).strip().lower()
    if s in ("true", "t", "1", "yes", "y"):
        return True
    if s in ("false", "f", "0", "no", "n"):
        return False
    return None


def _zfill_block(x):
    s = str(x)
    return s.zfill(3) if s.isdigit() else s


In [29]:
import numpy as np
import pandas as pd

def _pick_first_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _norm_block_key(x):
    # try to make a "best effort" key that matches whatever is in eye_trace_lookup
    # (many of your objects store block_num as int-like)
    s = str(x)
    try:
        return str(int(float(s)))
    except Exception:
        return s

def attach_other_eye_peak_speed(
    non_synced_df,
    eye_trace_lookup,
    t0_col="saccade_on_ms",
    eye_col="eye",
    animal_col="animal",
    block_col="block",
    ms_col="ms_axis",
    speed_col_candidates=None,
    window_ms=80,
    out_col="other_eye_peak_speed",
):
    """
    Adds out_col to non_synced_df:
      out_col = max(other_eye_speed) within [t0-window_ms, t0+window_ms]
    """
    if speed_col_candidates is None:
        # include the common ones I’ve seen across your codebases
        speed_col_candidates = [
            "angular_speed_r", "angular_speed", "speed_r", "speed",
            "velocity",  # BlockSync_current uses df['velocity'] for pupil speed:contentReference[oaicite:3]{index=3}
        ]

    d = non_synced_df.copy()
    peaks = []

    for _, r in d.iterrows():
        try:
            animal = r[animal_col]
            block = r[block_col]
            eye = str(r[eye_col]).upper()
            t0 = float(r[t0_col])
        except Exception:
            peaks.append(np.nan)
            continue

        other_eye = "R" if eye.startswith("L") else "L"

        # lookup key(s)
        k1 = (animal, block, other_eye)
        k2 = (str(animal), str(block), other_eye)
        k3 = (str(animal), _norm_block_key(block), other_eye)

        tr = None
        for k in (k1, k2, k3):
            if k in eye_trace_lookup:
                tr = eye_trace_lookup[k]
                break

        if tr is None or (ms_col not in tr.columns):
            peaks.append(np.nan)
            continue

        sp_col = _pick_first_col(tr, speed_col_candidates)
        if sp_col is None:
            peaks.append(np.nan)
            continue

        ms = pd.to_numeric(tr[ms_col], errors="coerce").to_numpy(dtype=float)
        sp = pd.to_numeric(tr[sp_col], errors="coerce").to_numpy(dtype=float)

        m = (ms >= t0 - float(window_ms)) & (ms <= t0 + float(window_ms)) & np.isfinite(sp)
        if not np.any(m):
            peaks.append(np.nan)
            continue

        peaks.append(float(np.nanmax(sp[m])))

    d[out_col] = peaks
    return d


def select_top_monocular_events(
    non_synced_saccade_collection,
    animal_call,
    block,
    eye_trace_lookup=None,          # <-- new
    window_ms_for_other_eye=80,     # <-- new
    n_examples=10,
    max_other_eye_metric=None,
    prefer_eye=None
):
    if isinstance(non_synced_saccade_collection, pd.DataFrame):
        df = non_synced_saccade_collection.copy()
    else:
        df = pd.DataFrame(non_synced_saccade_collection).copy()

    # required identifiers
    animal_col = _pick_first_col(df, ["animal_call", "animal", "_animal_std"])
    block_col  = _pick_first_col(df, ["block", "block_num", "_block_std"])
    eye_col    = _pick_first_col(df, ["eye", "_eye_std"])
    if animal_col is None or block_col is None or eye_col is None:
        raise KeyError("Missing one of required cols: animal, block, eye")

    # your real timing cols
    start_col = _pick_first_col(df, ["saccade_on_ms", "start_ms", "_start_ms_std", "start_time_ms"])
    end_col   = _pick_first_col(df, ["saccade_off_ms", "end_ms", "_end_ms_std", "end_time_ms"])
    if start_col is None or end_col is None:
        raise KeyError("Missing event timing cols (need saccade_on_ms / saccade_off_ms or equivalents)")

    amp_col = _pick_first_col(df, ["net_angular_disp", "net_disp_deg", "net_disp", "amplitude_deg", "amp_deg", "amp"])
    if amp_col is None:
        raise KeyError("No amplitude column found (expected net_angular_disp / amplitude_deg / etc.).")

    # If other-eye metric is missing, compute it
    other_col = _pick_first_col(df, [
        "other_eye_peak_speed", "other_eye_peak_vel", "other_eye_peak_velocity",
        "other_eye_max_speed", "other_eye_max_vel",
        "monocular_score", "other_eye_score"
    ])

    if other_col is None:
        if eye_trace_lookup is None:
            raise KeyError(
                "No 'other-eye motion' metric column exists, and no eye_trace_lookup was provided to compute it.\n"
                "Build eye_trace_lookup from block_collection, then call select_top_monocular_events(..., eye_trace_lookup=...)."
            )
        df = attach_other_eye_peak_speed(
            non_synced_df=df,
            eye_trace_lookup=eye_trace_lookup,
            t0_col=start_col,
            eye_col=eye_col,
            animal_col=animal_col,
            block_col=block_col,
            window_ms=window_ms_for_other_eye,
            out_col="other_eye_peak_speed",
        )
        other_col = "other_eye_peak_speed"

    # filter to single animal+block
    b_str = str(block)
    sub = df[(df[animal_col].astype(str) == str(animal_call)) &
             (df[block_col].astype(str) == b_str)].copy()

    # fallback: sometimes block stored as int-like in df
    if sub.empty:
        b2 = _norm_block_key(block)
        sub = df[(df[animal_col].astype(str) == str(animal_call)) &
                 (df[block_col].astype(str) == b2)].copy()

    if sub.empty:
        raise ValueError("No events found for animal={} block={}".format(animal_call, block))

    # eye filter
    if prefer_eye is not None:
        pe = str(prefer_eye).lower()
        if pe in ("l", "left"):
            sub = sub[sub[eye_col].astype(str).str.lower().str.startswith("l")].copy()
        elif pe in ("r", "right"):
            sub = sub[sub[eye_col].astype(str).str.lower().str.startswith("r")].copy()

    # numeric
    for c in (start_col, end_col, amp_col, other_col):
        sub[c] = pd.to_numeric(sub[c], errors="coerce")
    sub = sub.dropna(subset=[start_col, end_col, amp_col, other_col]).copy()

    if max_other_eye_metric is not None:
        sub = sub[sub[other_col] <= float(max_other_eye_metric)].copy()

    # rank: most monocular (low other-eye peak) + high amplitude
    sub = sub.sort_values([other_col, amp_col], ascending=[True, False]).copy()
    pool = sub.head(max(5 * int(n_examples), int(n_examples))).copy()
    pool = pool.sort_values([amp_col, other_col], ascending=[False, True]).head(int(n_examples)).reset_index(drop=True)

    # standardized cols for downstream UI
    pool["_animal_std"] = str(animal_call)
    pool["_block_std"] = str(block)
    pool["_start_ms_std"] = pool[start_col].astype(float)
    pool["_end_ms_std"] = pool[end_col].astype(float)
    pool["_eye_std"] = pool[eye_col].astype(str)

    view = pool[[animal_col, block_col, eye_col, start_col, end_col, amp_col, other_col]].copy()
    colmap = dict(animal_col=animal_col, block_col=block_col, eye_col=eye_col,
                  start_col=start_col, end_col=end_col, amp_col=amp_col, other_col=other_col)
    return pool, view, colmap


eye_trace_lookup = build_eye_trace_lookup_from_block_collection(block_collection)

ANIMAL = "PV_106"
BLOCK  = "011"

top_events_df, top_events_view, colmap = select_top_monocular_events(
    non_synced_saccade_collection,
    animal_call=ANIMAL,
    block=BLOCK,
    eye_trace_lookup=eye_trace_lookup,
    window_ms_for_other_eye=80,
    n_examples=10,
    max_other_eye_metric=None,
    prefer_eye=None
)

top_events_view


,animal,block,eye,saccade_on_ms,saccade_off_ms,net_angular_disp,other_eye_peak_speed
0,PV_106,011,L,1738025.15,1738091.75,3.801888,0.068240
1,PV_106,011,R,1421858.30,1421891.60,3.682619,0.056523
2,PV_106,011,L,1114665.80,1114699.10,3.487577,0.041838
3,PV_106,011,L,1956856.10,1956922.70,3.395845,0.049623
4,PV_106,011,R,1770076.40,1770143.00,3.261572,0.078489
5,PV_106,011,L,1114649.15,1114665.80,2.659038,0.041838
6,PV_106,011,L,741272.90,741306.20,2.585493,0.073441
7,PV_106,011,L,752245.25,752295.20,2.474284,0.052817
8,PV_106,011,L,1571525.15,1571575.10,2.369715,0.075919
9,PV_106,011,L,532981.40,533014.70,1.845653,0.057224


In [30]:
import numpy as np
import pandas as pd

def get_monocular_candidate_lists(
    non_synced_saccade_collection,
    animal_call,
    block,
    eye_trace_lookup=None,
    window_ms_for_other_eye=80,
    n_each=10,
    max_other_eye_metric=0.8,   # <-- IMPORTANT: set this to your monocular cutoff (units = your speed metric)
    prefer_eye=None
):
    """
    Returns two DataFrames:
      (A) most_monocular_df: top n_each with smallest other-eye motion (then high amp)
      (B) largest_amp_monocular_df: top n_each with largest amp among events that pass monocular criterion

    Requires:
      - timing cols: saccade_on_ms / saccade_off_ms
      - amplitude col: net_angular_disp (or similar)
      - other-eye metric: computed as other_eye_peak_speed if missing
    """

    # --- local helpers (reuse your earlier ones if already in notebook) ---
    def _pick_first_col(df, candidates):
        for c in candidates:
            if c in df.columns:
                return c
        return None

    def _norm_block_key(x):
        s = str(x)
        try:
            return str(int(float(s)))
        except Exception:
            return s

    def attach_other_eye_peak_speed(
        df_in,
        eye_trace_lookup,
        t0_col,
        eye_col,
        animal_col,
        block_col,
        ms_col="ms_axis",
        window_ms=80,
        out_col="other_eye_peak_speed",
        speed_col_candidates=None,
    ):
        if speed_col_candidates is None:
            speed_col_candidates = ["angular_speed_r","angular_speed","speed_r","speed","velocity"]

        d = df_in.copy()
        peaks = []

        for _, r in d.iterrows():
            try:
                animal = r[animal_col]
                blk = r[block_col]
                eye = str(r[eye_col]).upper()
                t0 = float(r[t0_col])
            except Exception:
                peaks.append(np.nan); continue

            other_eye = "R" if eye.startswith("L") else "L"

            keys = [
                (animal, blk, other_eye),
                (str(animal), str(blk), other_eye),
                (str(animal), _norm_block_key(blk), other_eye),
            ]
            tr = None
            for k in keys:
                if k in eye_trace_lookup:
                    tr = eye_trace_lookup[k]; break
            if tr is None or (ms_col not in tr.columns):
                peaks.append(np.nan); continue

            sp_col = _pick_first_col(tr, speed_col_candidates)
            if sp_col is None:
                peaks.append(np.nan); continue

            ms = pd.to_numeric(tr[ms_col], errors="coerce").to_numpy(dtype=float)
            sp = pd.to_numeric(tr[sp_col], errors="coerce").to_numpy(dtype=float)

            m = (ms >= t0 - float(window_ms)) & (ms <= t0 + float(window_ms)) & np.isfinite(sp)
            if not np.any(m):
                peaks.append(np.nan); continue

            peaks.append(float(np.nanmax(sp[m])))

        d[out_col] = peaks
        return d

    # --- input to df ---
    if isinstance(non_synced_saccade_collection, pd.DataFrame):
        df = non_synced_saccade_collection.copy()
    else:
        df = pd.DataFrame(non_synced_saccade_collection).copy()

    # --- columns ---
    animal_col = _pick_first_col(df, ["animal_call", "animal", "_animal_std"])
    block_col  = _pick_first_col(df, ["block", "block_num", "_block_std"])
    eye_col    = _pick_first_col(df, ["eye", "_eye_std"])
    if animal_col is None or block_col is None or eye_col is None:
        raise KeyError("Missing required identifiers: animal/block/eye.")

    start_col = _pick_first_col(df, ["saccade_on_ms", "start_ms", "_start_ms_std"])
    end_col   = _pick_first_col(df, ["saccade_off_ms", "end_ms", "_end_ms_std"])
    if start_col is None or end_col is None:
        raise KeyError("Missing timing cols (need saccade_on_ms and saccade_off_ms or equivalents).")

    amp_col = _pick_first_col(df, ["net_angular_disp","net_disp_deg","net_disp","amplitude_deg","amp_deg","amp"])
    if amp_col is None:
        raise KeyError("Missing amplitude col (need net_angular_disp or similar).")

    other_col = _pick_first_col(df, [
        "other_eye_peak_speed","other_eye_peak_vel","other_eye_peak_velocity",
        "other_eye_max_speed","other_eye_max_vel"
    ])

    # --- compute other-eye metric if absent ---
    if other_col is None:
        if eye_trace_lookup is None:
            raise KeyError("No other-eye metric in df, and eye_trace_lookup=None so I can't compute it.")
        df = attach_other_eye_peak_speed(
            df_in=df,
            eye_trace_lookup=eye_trace_lookup,
            t0_col=start_col,
            eye_col=eye_col,
            animal_col=animal_col,
            block_col=block_col,
            window_ms=window_ms_for_other_eye,
            out_col="other_eye_peak_speed"
        )
        other_col = "other_eye_peak_speed"

    # --- filter to block+animal ---
    b_str = str(block)
    sub = df[(df[animal_col].astype(str) == str(animal_call)) &
             (df[block_col].astype(str) == b_str)].copy()
    if sub.empty:
        b2 = _norm_block_key(block)
        sub = df[(df[animal_col].astype(str) == str(animal_call)) &
                 (df[block_col].astype(str) == b2)].copy()

    if sub.empty:
        raise ValueError("No events for animal={} block={}".format(animal_call, block))

    # --- optional eye filter ---
    if prefer_eye is not None:
        pe = str(prefer_eye).lower()
        if pe in ("l", "left"):
            sub = sub[sub[eye_col].astype(str).str.lower().str.startswith("l")].copy()
        elif pe in ("r", "right"):
            sub = sub[sub[eye_col].astype(str).str.lower().str.startswith("r")].copy()

    # --- numeric ---
    for c in (start_col, end_col, amp_col, other_col):
        sub[c] = pd.to_numeric(sub[c], errors="coerce")
    sub = sub.dropna(subset=[start_col, end_col, amp_col, other_col]).copy()

    # --- define "monocular pass" ---
    if max_other_eye_metric is None:
        # still allow ranking, but largest-amp list won't be strictly monocular
        monocular_pass = sub.copy()
    else:
        monocular_pass = sub[sub[other_col] <= float(max_other_eye_metric)].copy()

    # (A) most monocular: smallest other-eye metric, then large amp
    most_monocular = sub.sort_values([other_col, amp_col], ascending=[True, False]).head(int(n_each)).copy()

    # (B) largest amplitude monocular: among pass set, max amplitude, tie-break by other-eye metric
    if monocular_pass.empty:
        largest_amp_monocular = monocular_pass.copy()  # empty
    else:
        largest_amp_monocular = monocular_pass.sort_values([amp_col, other_col], ascending=[False, True]).head(int(n_each)).copy()

    # --- standardize cols for downstream viewer ---
    for out_df in (most_monocular, largest_amp_monocular):
        out_df["_animal_std"] = str(animal_call)
        out_df["_block_std"] = str(block)
        out_df["_start_ms_std"] = out_df[start_col].astype(float)
        out_df["_end_ms_std"] = out_df[end_col].astype(float)
        out_df["_eye_std"] = out_df[eye_col].astype(str)

    # compact views
    view_cols = [animal_col, block_col, eye_col, start_col, end_col, amp_col, other_col]
    view_cols = [c for c in view_cols if c in sub.columns]

    most_view = most_monocular[view_cols].reset_index(drop=True)
    amp_view  = largest_amp_monocular[view_cols].reset_index(drop=True)

    meta = dict(
        animal_col=animal_col, block_col=block_col, eye_col=eye_col,
        start_col=start_col, end_col=end_col, amp_col=amp_col, other_col=other_col,
        max_other_eye_metric=max_other_eye_metric
    )
    return most_monocular, largest_amp_monocular, most_view, amp_view, meta


In [42]:
# Build trace lookup once (from your existing pipeline helper)
# eye_trace_lookup = build_eye_trace_lookup_from_block_collection(block_collection)

ANIMAL = "PV_106"
BLOCK = "011"

# Pick a real cutoff. If your "speed" metric is deg/frame, 0.8 deg/frame ~ 48 deg/s at 60 fps.
MAX_OTHER_EYE = 0.8

most_df, largest_amp_df, most_view, amp_view, meta = get_monocular_candidate_lists(
    non_synced_saccade_collection,
    animal_call=ANIMAL,
    block=BLOCK,
    eye_trace_lookup=eye_trace_lookup,
    window_ms_for_other_eye=80,
    n_each=50,
    max_other_eye_metric=MAX_OTHER_EYE,
    prefer_eye=None
)

print("=== Most monocular (top 10) ===")
display(most_view)

print("=== Largest amplitude monocular (top 10) ===")
display(amp_view)


=== Most monocular (top 10) ===


,animal,block,eye,saccade_on_ms,saccade_off_ms,net_angular_disp,other_eye_peak_speed
0,PV_106,011,L,1562733.95,1562750.60,0.831717,0.038099
1,PV_106,011,L,191140.25,191156.90,0.837499,0.040174
2,PV_106,011,L,191156.90,191173.55,0.802887,0.040174
3,PV_106,011,L,1081981.85,1081998.50,0.891567,0.041162
4,PV_106,011,L,1114665.80,1114699.10,3.487577,0.041838
5,PV_106,011,L,1114649.15,1114665.80,2.659038,0.041838
6,PV_106,011,L,1132298.15,1132331.45,0.874732,0.046816
7,PV_106,011,L,2018744.15,2018760.80,0.673400,0.047572
8,PV_106,011,R,1421941.55,1421974.85,1.373206,0.048792
9,PV_106,011,L,1956856.10,1956922.70,3.395845,0.049623


=== Largest amplitude monocular (top 10) ===


,animal,block,eye,saccade_on_ms,saccade_off_ms,net_angular_disp,other_eye_peak_speed
0,PV_106,011,L,649548.05,649697.90,34.823644,0.457379
1,PV_106,011,L,1397432.75,1397582.60,21.972547,0.178496
2,PV_106,011,L,664583.00,664649.60,18.735991,0.539145
3,PV_106,011,L,745751.75,745918.25,17.760907,0.467697
4,PV_106,011,R,293354.60,293454.50,16.237197,0.721173
5,PV_106,011,R,1821008.75,1821125.30,14.218852,0.156091
6,PV_106,011,L,394769.75,394836.35,14.215709,0.583258
7,PV_106,011,L,1316097.50,1316164.10,14.002508,0.526883
8,PV_106,011,L,528169.55,528252.80,13.935054,0.567019
9,PV_106,011,L,552178.85,552228.80,13.903795,0.296782


In [49]:
# --- Monocular verification: OpenCV manual curation UI (sterile, stand-alone) ---
import os
import re
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime

def curate_events_for_monocular_video_opencv(
    block_dict,
    events_df,
    animal_call=None,
    block=None,
    pre_ms=200.0,
    post_ms=200.0,
    flip_eyes=True,
    show_arena=False,
    text_cols=None,
    # display scaling:
    window_scale=0.60,
    scale_step=0.10,
    min_scale=0.20,
    max_scale=2.00,
    # playback:
    wait_ms_default=15,          # ~60–70 Hz redraw; independent of step_ms
    wait_step=5,
    min_wait_ms=1,
    max_wait_ms=200,
    # tagging (sterile)
    tag_col="keep_for_video",    # True=keep, False=reject, None/unset
    ts_col="curation_ts",
    out_dir_name="monocular_video_curation",
    overwrite_existing=False
):
    """
    Manual curation UI for monocular events.
    - events_df: can be your 'top events view' or full df; must contain animal+block+eye+start/end info.
    - Tags saved per block to: bs.analysis_path / out_dir_name / '{animal}_block_{block}_monocular_curation.csv'

    UI (mouse buttons + keys):
      Space: play/pause
      [ / ]: prev / next event
      , / .: step -1 / +1 frame (in time, using step_ms)
      k: KEEP (tag_col=True)
      r: REJECT (tag_col=False)
      e: export per-block CSVs (sterile)
      z / x: scale - / scale +
      - / +: slower / faster (changes wait_ms)
      q / Esc: quit

    Returns: events dataframe with tag_col + ts_col.
    """

    if text_cols is None:
        text_cols = []  # optionally show columns from eye dfs on overlay

    # ---------------- helpers ----------------
    def _now_stamp():
        return datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    def _zfill_block(b):
        s = str(b)
        m = re.search(r"(\d+)", s)
        if m:
            return m.group(1).zfill(3)
        return s.zfill(3)

    def _first_existing(df, candidates):
        for c in candidates:
            if c in df.columns:
                return c
        return None

    def _require_col(df, candidates, label):
        c = _first_existing(df, candidates)
        if c is None:
            raise KeyError("Could not find {}. Tried: {}".format(label, candidates))
        return c

    def _coerce_tag(v):
        # allow None/NaN -> None; else bool
        if v is None:
            return None
        try:
            if isinstance(v, float) and np.isnan(v):
                return None
        except Exception:
            pass
        return bool(v)

    def _block_for_key(block_dict_local, animal, b):
        """
        Supports:
          - dict keys as (animal, block)
          - dict keys as 'PV_106_block_011'
          - fallback: search string keys containing animal and block token.
        """
        b = _zfill_block(b)
        animal = str(animal)

        # tuple key
        if (animal, b) in block_dict_local:
            return block_dict_local[(animal, b)]

        # string key (your common case)
        k1 = "{}_block_{}".format(animal, b)
        if k1 in block_dict_local:
            return block_dict_local[k1]

        # sometimes block like 'block_011' inside key
        k2 = "{}_{}".format(animal, "block_{}".format(b))
        if k2 in block_dict_local:
            return block_dict_local[k2]

        # brute search on string keys
        for k in block_dict_local.keys():
            if isinstance(k, str):
                if (animal in k) and ("block_{}".format(b) in k or k.endswith("_{}".format(b))):
                    return block_dict_local[k]

        raise KeyError("Could not find BlockSync for animal={} block={}".format(animal, b))

    def _get_eye_dfs(bs):
        # be permissive to your attribute naming across versions
        left_df = getattr(bs, "left_eye_data", None)
        right_df = getattr(bs, "right_eye_data", None)
        if left_df is None:
            left_df = getattr(bs, "le_df", None)
        if right_df is None:
            right_df = getattr(bs, "re_df", None)
        return left_df, right_df

    def _get_video_paths(bs):
        # common in your pipeline
        lv = getattr(bs, "le_videos", None)
        rv = getattr(bs, "re_videos", None)
        av = getattr(bs, "arena_videos", None)

        lpath = None
        rpath = None
        apaths = []

        if isinstance(lv, (list, tuple)) and len(lv):
            lpath = lv[0]
        elif isinstance(lv, str):
            lpath = lv

        if isinstance(rv, (list, tuple)) and len(rv):
            rpath = rv[0]
        elif isinstance(rv, str):
            rpath = rv

        if isinstance(av, (list, tuple)):
            apaths = list(av)
        elif isinstance(av, str):
            apaths = [av]

        return lpath, rpath, apaths

    def _frame_col(df_eye):
        # try your common frame columns
        for c in ["eye_frame", "L_eye_frame", "R_eye_frame", "frame", "frame_idx"]:
            if c in df_eye.columns:
                return c
        return None

    def _nearest_row(df_eye, t_ms):
        if df_eye is None or df_eye.empty:
            return None
        if "ms_axis" not in df_eye.columns:
            return None
        ms = df_eye["ms_axis"].values
        if ms.size == 0:
            return None
        idx = int(np.searchsorted(ms, t_ms))
        if idx <= 0:
            return df_eye.iloc[0]
        if idx >= ms.size:
            return df_eye.iloc[-1]
        # choose closer neighbor
        if abs(ms[idx] - t_ms) < abs(ms[idx - 1] - t_ms):
            return df_eye.iloc[idx]
        return df_eye.iloc[idx - 1]

    def _median_step_ms(df_eye):
        if df_eye is None or df_eye.empty or "ms_axis" not in df_eye.columns:
            return 1000.0 / 60.0
        ms = df_eye["ms_axis"].values
        if ms.size < 2:
            return 1000.0 / 60.0
        d = np.diff(ms)
        d = d[np.isfinite(d) & (d > 0)]
        if d.size == 0:
            return 1000.0 / 60.0
        return float(np.median(d))

    def _apply_flip(img):
        if not flip_eyes:
            return img
        img = cv2.flip(img, 0)
        img = cv2.flip(img, 1)
        return img

    def _overlay_text(img, lines, origin=(10, 24), vstep=22, color=(255, 255, 255), font_scale=0.6, thickness=1):
        x, y = origin
        for ln in lines:
            cv2.putText(img, ln, (x, y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)
            y += vstep

    def _overlay_ellipse(img, df_eye, frame_idx, thickness=2):
        # draws pupil ellipse if columns exist
        if df_eye is None or frame_idx is None:
            return
        fc = _frame_col(df_eye)
        if fc is None:
            return
        hit = df_eye[df_eye[fc] == frame_idx]
        if hit.empty:
            return
        row = hit.iloc[0]
        cx = row.get("center_x", np.nan)
        cy = row.get("center_y", np.nan)
        w  = row.get("width", np.nan)
        h  = row.get("height", np.nan)
        phi = row.get("phi", np.nan)
        if not (pd.isna(cx) or pd.isna(cy) or pd.isna(w) or pd.isna(h)):
            cv2.ellipse(
                img,
                (int(round(cx)), int(round(cy))),
                (max(1, int(round(w))), max(1, int(round(h)))),
                float(0 if pd.isna(phi) else phi),
                0, 360, (0, 255, 0),
                thickness
            )

    def _ensure_std_columns(df_local, animal_col, block_col, eye_col, start_col, end_col):
        df_local["_animal_std"] = df_local[animal_col].astype(str)
        df_local["_block_std"]  = df_local[block_col].astype(str).apply(_zfill_block)
        df_local["_eye_std"]    = df_local[eye_col].astype(str)
        df_local["_start_ms_std"] = pd.to_numeric(df_local[start_col], errors="coerce")
        df_local["_end_ms_std"]   = pd.to_numeric(df_local[end_col], errors="coerce")

        # extend window for video review
        df_local["_win_start_ms"] = df_local["_start_ms_std"] - float(pre_ms)
        df_local["_win_end_ms"]   = df_local["_end_ms_std"] + float(post_ms)

        if tag_col not in df_local.columns:
            df_local[tag_col] = None
        if ts_col not in df_local.columns:
            df_local[ts_col] = ""

        return df_local

    def _write_block_annotations(bs, sub_df):
        out_dir = Path(bs.analysis_path) / out_dir_name
        out_dir.mkdir(parents=True, exist_ok=True)

        animal_s = str(sub_df["_animal_std"].iloc[0])
        block_s  = str(sub_df["_block_std"].iloc[0])
        out_path = out_dir / "{}_block_{}_monocular_curation.csv".format(animal_s, block_s)

        out = pd.DataFrame({
            "animal_call": sub_df["_animal_std"].astype(str),
            "block": sub_df["_block_std"].astype(str),
            "eye": sub_df["_eye_std"].astype(str),
            "start_ms": sub_df["_start_ms_std"].astype(float),
            "end_ms": sub_df["_end_ms_std"].astype(float),
            tag_col: sub_df[tag_col].map(_coerce_tag).astype("object"),
            ts_col: sub_df[ts_col].astype(str)
        })

        # merge with existing (unless overwrite)
        if out_path.exists() and (not overwrite_existing):
            try:
                old = pd.read_csv(out_path)
                # outer merge on identity columns
                key = ["animal_call", "block", "eye", "start_ms", "end_ms"]
                merged = old.merge(out, on=key, how="outer", suffixes=("_old", ""))
                # prefer new tag/ts if present
                if "{}_old".format(tag_col) in merged.columns:
                    merged[tag_col] = merged[tag_col].where(merged[tag_col].notna(), merged["{}_old".format(tag_col)])
                    merged = merged.drop(columns=["{}_old".format(tag_col)], errors="ignore")
                if "{}_old".format(ts_col) in merged.columns:
                    merged[ts_col] = merged[ts_col].where(merged[ts_col].astype(str) != "", merged["{}_old".format(ts_col)].astype(str))
                    merged = merged.drop(columns=["{}_old".format(ts_col)], errors="ignore")
                merged.to_csv(out_path, index=False)
                return out_path
            except Exception:
                # if merge fails, fall back to writing new
                out.to_csv(out_path, index=False)
                return out_path

        out.to_csv(out_path, index=False)
        return out_path

    # ---------------- normalize events df ----------------
    if isinstance(events_df, pd.DataFrame):
        df = events_df.copy()
    else:
        df = pd.DataFrame(events_df).copy()

    # allow filtering by animal/block if requested
    animal_col = _require_col(df, ["animal_call", "animal", "_animal_std"], "animal column")
    block_col  = _require_col(df, ["block", "block_num", "_block_std"], "block column")
    eye_col    = _require_col(df, ["eye", "_eye_std"], "eye column")

    start_col = _require_col(df, ["_start_ms_std", "start_ms", "start_time_ms", "saccade_on_ms"], "start_ms column")
    end_col   = _require_col(df, ["_end_ms_std", "end_ms", "end_time_ms", "saccade_off_ms"], "end_ms column")

    df = _ensure_std_columns(df, animal_col, block_col, eye_col, start_col, end_col)
    df = df.dropna(subset=["_start_ms_std", "_end_ms_std"]).reset_index(drop=True)

    if animal_call is not None:
        df = df[df["_animal_std"].astype(str) == str(animal_call)].copy()
    if block is not None:
        df = df[df["_block_std"].astype(str) == _zfill_block(block)].copy()

    if df.empty:
        raise ValueError("No events to curate after filtering (animal_call/block).")

    # ---------------- OpenCV UI ----------------
    # windows (mirror your pattern: separate windows + Controls):contentReference[oaicite:5]{index=5}
    cv2.namedWindow("Controls", cv2.WINDOW_NORMAL)
    cv2.namedWindow("Left Eye", cv2.WINDOW_NORMAL)
    cv2.namedWindow("Right Eye", cv2.WINDOW_NORMAL)
    if show_arena:
        cv2.namedWindow("Arena", cv2.WINDOW_NORMAL)

    # Controls panel geometry + buttons (your style):contentReference[oaicite:6]{index=6}
    ctrl_w, ctrl_h = 540, 520
    buttons = {
        "Play":      ((10, 10),   (260, 60)),
        "Pause":     ((280, 10),  (530, 60)),
        "Prev":      ((10, 80),   (260, 130)),
        "Next":      ((280, 80),  (530, 130)),
        "Step -1":   ((10, 150),  (260, 200)),
        "Step +1":   ((280, 150), (530, 200)),
        "KEEP":      ((10, 220),  (260, 270)),
        "REJECT":    ((280, 220), (530, 270)),
        "Scale -":   ((10, 290),  (260, 340)),
        "Scale +":   ((280, 290), (530, 340)),
        "Slower":    ((10, 360),  (260, 410)),
        "Faster":    ((280, 360), (530, 410)),
        "Export":    ((10, 430),  (530, 480)),
    }

    COLOR_BG = (60, 60, 60)
    COLOR_BORDER = (180, 180, 180)
    COLOR_TEXT = (220, 220, 220)
    COLOR_KEEP = (0, 255, 0)
    COLOR_REJ  = (0, 0, 255)
    COLOR_INFO = (180, 255, 180)
    COLOR_WARN = (0, 165, 255)

    playing = False
    quit_flag = False
    cur_idx = 0
    cur_ms = None
    step_ms = 1000.0 / 60.0
    wait_ms = int(wait_ms_default)
    last_status = ""

    # video handles (persist across events when same block)
    capL = None
    capR = None
    capA = None
    arena_idx = 0
    current_key = (None, None)

    # current eye dfs / frame cols / dims
    left_df = None
    right_df = None
    left_fc = None
    right_fc = None
    arena_df = None
    arena_fc = None

    # display dims (computed on open)
    Wl = Hl = Wr = Hr = Wa = Ha = 0
    disp_Wl = disp_Hl = disp_Wr = disp_Hr = disp_Wa = disp_Ha = 0

    def _hit_button(x, y):
        for name, ((x1, y1), (x2, y2)) in buttons.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                return name
        return None

    def _draw_controls(idx):
        img = np.zeros((ctrl_h, ctrl_w, 3), dtype=np.uint8)
        # header
        state = df[tag_col].iloc[idx]
        state = _coerce_tag(state)
        if state is None:
            state_str = "UNSET"
        elif bool(state):
            state_str = "KEEP"
        else:
            state_str = "REJECT"

        header = "Event {}/{} | state={} | scale={:.2f} | wait={}ms".format(
            idx + 1, len(df), state_str, float(window_scale), int(wait_ms)
        )
        cv2.putText(img, header, (10, ctrl_h - 12), cv2.FONT_HERSHEY_SIMPLEX, 0.55, COLOR_TEXT, 1, cv2.LINE_AA)

        # status line
        if last_status:
            cv2.putText(img, last_status, (10, 22), cv2.FONT_HERSHEY_SIMPLEX, 0.55, COLOR_INFO, 1, cv2.LINE_AA)

        # buttons
        for name, ((x1, y1), (x2, y2)) in buttons.items():
            # color by function/state
            color = COLOR_BORDER
            if name == "KEEP":
                color = COLOR_KEEP
            elif name == "REJECT":
                color = COLOR_REJ
            elif name == "Export":
                color = COLOR_WARN
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, name, (x1 + 10, y2 - 18), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLOR_TEXT, 2, cv2.LINE_AA)
        return img

    def _seek(cap, idx):
        cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, int(idx)))

    def _read(cap):
        ret, f = cap.read()
        return f if ret else None

    def _set_display_scale(force=False):
        nonlocal disp_Wl, disp_Hl, disp_Wr, disp_Hr, disp_Wa, disp_Ha
        nonlocal window_scale
        # compute & resizeWindow (your pattern):contentReference[oaicite:7]{index=7}
        if Wl > 0 and Hl > 0:
            disp_Wl, disp_Hl = int(Wl * window_scale), int(Hl * window_scale)
            cv2.resizeWindow("Left Eye", disp_Wl, disp_Hl)
        if Wr > 0 and Hr > 0:
            disp_Wr, disp_Hr = int(Wr * window_scale), int(Hr * window_scale)
            cv2.resizeWindow("Right Eye", disp_Wr, disp_Hr)
        if show_arena and Wa > 0 and Ha > 0:
            disp_Wa, disp_Ha = int(Wa * window_scale), int(Ha * window_scale)
            cv2.resizeWindow("Arena", disp_Wa, disp_Ha)
        cv2.resizeWindow("Controls", ctrl_w, ctrl_h)

    def _open_for(animal, b):
        nonlocal capL, capR, capA, current_key, arena_idx
        nonlocal left_df, right_df, left_fc, right_fc, Wl, Hl, Wr, Hr, Wa, Ha
        nonlocal arena_df, arena_fc

        key = (str(animal), _zfill_block(b))
        if key == current_key and capL is not None and capR is not None:
            return

        # close previous
        try:
            if capL is not None: capL.release()
            if capR is not None: capR.release()
            if capA is not None: capA.release()
        except Exception:
            pass

        bs = _block_for_key(block_dict, key[0], key[1])
        left_df, right_df = _get_eye_dfs(bs)
        left_fc = _frame_col(left_df) if left_df is not None else None
        right_fc = _frame_col(right_df) if right_df is not None else None

        lpath, rpath, apaths = _get_video_paths(bs)
        if lpath is None or rpath is None:
            raise RuntimeError("Could not resolve le/re video paths for {} block {}".format(key[0], key[1]))

        capL = cv2.VideoCapture(str(lpath))
        capR = cv2.VideoCapture(str(rpath))
        if not capL.isOpened() or not capR.isOpened():
            raise RuntimeError("Could not open eye videos: L={} R={}".format(lpath, rpath))

        # dims
        Wl = int(capL.get(cv2.CAP_PROP_FRAME_WIDTH))
        Hl = int(capL.get(cv2.CAP_PROP_FRAME_HEIGHT))
        Wr = int(capR.get(cv2.CAP_PROP_FRAME_WIDTH))
        Hr = int(capR.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # arena
        arena_df = None
        arena_fc = None
        Wa = Ha = 0
        capA = None
        arena_idx = 0
        if show_arena and apaths:
            capA_try = cv2.VideoCapture(str(apaths[arena_idx]))
            if capA_try.isOpened():
                capA = capA_try
                Wa = int(capA.get(cv2.CAP_PROP_FRAME_WIDTH))
                Ha = int(capA.get(cv2.CAP_PROP_FRAME_HEIGHT))
                # if you have a precomputed arena sync df, plug here; otherwise leave None.
                # For now: no arena_df => "no synchronized frame" message.
                arena_df = None
                arena_fc = None
            else:
                capA = None

        current_key = key
        _set_display_scale(force=True)

    def _export_per_block():
        nonlocal last_status
        written = []
        # group by standardized (animal, block)
        for (animal, b), sub in df.groupby(["_animal_std", "_block_std"], dropna=False):
            bs = _block_for_key(block_dict, animal, b)
            p = _write_block_annotations(bs, sub.copy())
            written.append(str(p))
        last_status = "Exported sterile curation to {} block file(s).".format(len(written))

    # initial paint
    cv2.imshow("Controls", _draw_controls(cur_idx))

    def on_mouse_controls(event, x, y, flags, param):
        nonlocal playing, quit_flag, cur_idx, cur_ms, last_status, window_scale, wait_ms
        if event != cv2.EVENT_LBUTTONDOWN:
            return
        name = _hit_button(x, y)
        if name is None:
            return

        last_status = ""

        if name == "Play":
            playing = True
        elif name == "Pause":
            playing = False
        elif name == "Prev":
            playing = False
            cur_idx = (cur_idx - 1) % len(df)
            cur_ms = None
        elif name == "Next":
            playing = False
            cur_idx = (cur_idx + 1) % len(df)
            cur_ms = None
        elif name == "Step -1":
            playing = False
            if cur_ms is not None:
                cur_ms -= step_ms
        elif name == "Step +1":
            playing = False
            if cur_ms is not None:
                cur_ms += step_ms
        elif name == "KEEP":
            df.at[cur_idx, tag_col] = True
            df.at[cur_idx, ts_col] = _now_stamp()
        elif name == "REJECT":
            df.at[cur_idx, tag_col] = False
            df.at[cur_idx, ts_col] = _now_stamp()
        elif name == "Scale -":
            window_scale = max(min_scale, float(window_scale) - float(scale_step))
            _set_display_scale(force=True)
        elif name == "Scale +":
            window_scale = min(max_scale, float(window_scale) + float(scale_step))
            _set_display_scale(force=True)
        elif name == "Slower":
            wait_ms = min(max_wait_ms, int(wait_ms) + int(wait_step))
        elif name == "Faster":
            wait_ms = max(min_wait_ms, int(wait_ms) - int(wait_step))
        elif name == "Export":
            _export_per_block()

        cv2.imshow("Controls", _draw_controls(cur_idx))

    cv2.setMouseCallback("Controls", on_mouse_controls)

    # ---------------- main loop ----------------
    while True:
        k = cv2.waitKey(int(wait_ms)) & 0xFF

        # keyboard shortcuts mirror your original structure:contentReference[oaicite:8]{index=8}
        if k in (27, ord('q'), ord('Q')):
            quit_flag = True
        elif k == 32:  # space
            playing = not playing
        elif k == ord('['):
            playing = False
            cur_idx = (cur_idx - 1) % len(df)
            cur_ms = None
        elif k == ord(']'):
            playing = False
            cur_idx = (cur_idx + 1) % len(df)
            cur_ms = None
        elif k == ord(','):
            playing = False
            if cur_ms is not None:
                cur_ms -= step_ms
        elif k == ord('.'):
            playing = False
            if cur_ms is not None:
                cur_ms += step_ms
        elif k in (ord('k'), ord('K')):
            df.at[cur_idx, tag_col] = True
            df.at[cur_idx, ts_col] = _now_stamp()
        elif k in (ord('r'), ord('R')):
            df.at[cur_idx, tag_col] = False
            df.at[cur_idx, ts_col] = _now_stamp()
        elif k in (ord('e'), ord('E')):
            _export_per_block()
        elif k in (ord('z'), ord('Z')):
            window_scale = max(min_scale, float(window_scale) - float(scale_step))
            _set_display_scale(force=True)
        elif k in (ord('x'), ord('X')):
            window_scale = min(max_scale, float(window_scale) + float(scale_step))
            _set_display_scale(force=True)
        elif k == ord('-'):
            wait_ms = min(max_wait_ms, int(wait_ms) + int(wait_step))
        elif k in (ord('+'), ord('=')):  # '=' is '+' without shift on many keyboards
            wait_ms = max(min_wait_ms, int(wait_ms) - int(wait_step))

        if quit_flag:
            break

        row = df.iloc[cur_idx]
        animal = str(row["_animal_std"])
        b = str(row["_block_std"])

        # open block
        _open_for(animal, b)

        # update step_ms from actual eye df timing
        step_ms = float(np.mean([_median_step_ms(left_df), _median_step_ms(right_df)]))

        start_ms = float(row["_win_start_ms"])
        end_ms   = float(row["_win_end_ms"])

        if cur_ms is None:
            cur_ms = start_ms
        cur_ms = min(max(cur_ms, start_ms), end_ms)

        # map to nearest rows in eye dfs
        rowL = _nearest_row(left_df, cur_ms)
        rowR = _nearest_row(right_df, cur_ms)

        # Left frame
        L_img = np.zeros((max(1, Hl), max(1, Wl), 3), dtype=np.uint8)
        if rowL is not None and left_fc is not None:
            L_idx = rowL.get(left_fc, np.nan)
            if pd.notna(L_idx):
                L_idx = int(L_idx)
                _seek(capL, L_idx)
                fL = _read(capL)
                if fL is not None:
                    _overlay_ellipse(fL, left_df, L_idx)
                    img = _apply_flip(fL.copy())
                    lines = ["Left | {} B{} | t={:.1f}ms | frame={}".format(animal, b, cur_ms, L_idx)]
                    for c in text_cols:
                        if (left_df is not None) and (c in left_df.columns):
                            v = rowL.get(c, np.nan)
                            if pd.notna(v):
                                try:
                                    lines.append("{}={:.3f}".format(c, float(v)))
                                except Exception:
                                    pass
                    _overlay_text(img, lines, origin=(10, 24))
                    L_img = img
                else:
                    _overlay_text(L_img, ["Left | read error"], origin=(10, 24))
                    L_img = _apply_flip(L_img)
            else:
                _overlay_text(L_img, ["Left | no synchronized frame"], origin=(10, 24))
                L_img = _apply_flip(L_img)

        # Right frame
        R_img = np.zeros((max(1, Hr), max(1, Wr), 3), dtype=np.uint8)
        if rowR is not None and right_fc is not None:
            R_idx = rowR.get(right_fc, np.nan)
            if pd.notna(R_idx):
                R_idx = int(R_idx)
                _seek(capR, R_idx)
                fR = _read(capR)
                if fR is not None:
                    _overlay_ellipse(fR, right_df, R_idx)
                    img = _apply_flip(fR.copy())
                    lines = ["Right | {} B{} | t={:.1f}ms | frame={}".format(animal, b, cur_ms, R_idx)]
                    for c in text_cols:
                        if (right_df is not None) and (c in right_df.columns):
                            v = rowR.get(c, np.nan)
                            if pd.notna(v):
                                try:
                                    lines.append("{}={:.3f}".format(c, float(v)))
                                except Exception:
                                    pass
                    _overlay_text(img, lines, origin=(10, 24))
                    R_img = img
                else:
                    _overlay_text(R_img, ["Right | read error"], origin=(10, 24))
                    R_img = _apply_flip(R_img)
            else:
                _overlay_text(R_img, ["Right | no synchronized frame"], origin=(10, 24))
                R_img = _apply_flip(R_img)

        # show
        if disp_Wl > 0 and disp_Hl > 0:
            cv2.imshow("Left Eye", cv2.resize(L_img, (disp_Wl, disp_Hl)))
        else:
            cv2.imshow("Left Eye", L_img)

        if disp_Wr > 0 and disp_Hr > 0:
            cv2.imshow("Right Eye", cv2.resize(R_img, (disp_Wr, disp_Hr)))
        else:
            cv2.imshow("Right Eye", R_img)

        # controls repaint
        cv2.imshow("Controls", _draw_controls(cur_idx))

        # playback advance (same logic as your outlier tool):contentReference[oaicite:9]{index=9}
        if playing:
            cur_ms += step_ms
            if cur_ms > end_ms:
                playing = False
                cur_idx = (cur_idx + 1) % len(df)
                cur_ms = None

    # cleanup
    try:
        if capL is not None: capL.release()
        if capR is not None: capR.release()
        if capA is not None: capA.release()
    except Exception:
        pass
    cv2.destroyAllWindows()

    return df


df = curate_events_for_monocular_video_opencv(
    block_dict,
    largest_amp_df,
    animal_call="PV_106",
    block="011",
    pre_ms=250.0,
    post_ms=250.0,
    flip_eyes=True,
    show_arena=True,
    text_cols=None,
    # display scaling:
    window_scale=0.60,
    scale_step=0.10,
    min_scale=0.20,
    max_scale=2.00,
    # playback:
    wait_ms_default=15,          # ~60–70 Hz redraw; independent of step_ms
    wait_step=5,
    min_wait_ms=1,
    max_wait_ms=200,
    # tagging (sterile)
    tag_col="keep_for_video",    # True=keep, False=reject, None/unset
    ts_col="curation_ts",
    out_dir_name="monocular_video_curation",
    overwrite_existing=False
)

In [50]:
df

,saccade_start_ind,saccade_end_ind,saccade_start_timestamp,saccade_end_timestamp,saccade_on_ms,saccade_off_ms,length,magnitude_raw_pixel,magnitude_pixel,magnitude_raw_angular,...,other_eye_peak_speed,_animal_std,_block_std,_start_ms_std,_end_ms_std,_eye_std,_win_start_ms,_win_end_ms,keep_for_video,curation_ts
0,37609,37618,12990961.0,12993958.0,649548.05,649697.90,9,74.325777,74.325777,41.100213,...,0.457379,PV_106,011,649548.05,649697.90,L,649298.05,649947.90,True,2025-12-31_17-22-25
1,82527,82536,27948655.0,27951652.0,1397432.75,1397582.60,9,57.118510,57.118510,29.949464,...,0.178496,PV_106,011,1397432.75,1397582.60,L,1397182.75,1397832.60,False,2025-12-31_17-22-29
2,38512,38516,13291660.0,13292992.0,664583.00,664649.60,4,22.086375,22.086375,23.127384,...,0.539145,PV_106,011,664583.00,664649.60,L,664333.00,664899.60,True,2025-12-31_17-22-30
3,43387,43397,14915035.0,14918365.0,745751.75,745918.25,10,32.182984,32.182984,19.086935,...,0.467697,PV_106,011,745751.75,745918.25,L,745501.75,746168.25,True,2025-12-31_17-22-38
4,16216,16222,5867092.0,5869090.0,293354.60,293454.50,6,38.189611,38.189611,18.225718,...,0.721173,PV_106,011,293354.60,293454.50,R,293104.60,293704.50,True,2025-12-31_17-22-41
5,107967,107974,36420175.0,36422506.0,1821008.75,1821125.30,7,38.978089,38.978089,16.010049,...,0.156091,PV_106,011,1821008.75,1821125.30,R,1820758.75,1821375.30,True,2025-12-31_17-22-46
6,22307,22311,7895395.0,7896727.0,394769.75,394836.35,4,29.517137,29.517137,15.847297,...,0.583258,PV_106,011,394769.75,394836.35,L,394519.75,395086.35,False,2025-12-31_17-22-49
7,77642,77646,26321950.0,26323282.0,1316097.50,1316164.10,4,31.443291,31.443291,18.284548,...,0.526883,PV_106,011,1316097.50,1316164.10,L,1315847.50,1316414.10,False,2025-12-31_17-22-51
8,30319,30324,10563391.0,10565056.0,528169.55,528252.80,5,33.783532,33.783532,18.780982,...,0.567019,PV_106,011,528169.55,528252.80,L,527919.55,528502.80,False,2025-12-31_17-23-05
9,31761,31764,11043577.0,11044576.0,552178.85,552228.80,3,28.881512,28.881512,20.099988,...,0.296782,PV_106,011,552178.85,552228.80,L,551928.85,552478.80,True,2025-12-31_17-23-18


In [52]:
# =========================
# Export curated events to MP4 (stand-alone cell, py<3.9)
# =========================
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

def _zfill_block(block):
    s = str(block)
    digits = "".join([c for c in s if c.isdigit()])
    if digits == "":
        return s.zfill(3)
    return digits.zfill(3)

def _block_for_key(block_dict, animal, block):
    """
    Supports your dict style keys: 'PV_106_block_011'
    """
    b = _zfill_block(block)
    k1 = "{}_block_{}".format(str(animal), b)
    if k1 in block_dict:
        return block_dict[k1]

    # (fallbacks if you ever also use tuple keys)
    if (str(animal), b) in block_dict:
        return block_dict[(str(animal), b)]
    if (str(animal), str(int(b))) in block_dict:
        return block_dict[(str(animal), str(int(b)))]

    raise KeyError("Could not find BlockSync for animal={} block={}".format(animal, b))

def _frame_col(df):
    for c in ("eye_frame", "frame", "frame_idx", "video_frame"):
        if c in df.columns:
            return c
    return None

def _nearest_row(df, ms):
    arr = df["ms_axis"].values
    if arr.size == 0:
        return None
    idx = int(np.argmin(np.abs(arr - float(ms))))
    return df.iloc[idx]

def _ensure_arena_ms_axis(bs):
    """
    Uses bs.final_sync_df; if it lacks ms_axis, tries to merge from eye ms_axis via OE_timestamp.
    Mirrors your reviewer logic.:contentReference[oaicite:2]{index=2}
    """
    fs = bs.final_sync_df.copy()
    # resolve arena frame col
    fcol = None
    for c in ["Arena_frame", "arena_frame", "arena_frames", "arena_frame_idx", "frame", "frame_idx", "video_frame", "arena_idx"]:
        if c in fs.columns:
            fcol = c
            break
    if fcol is None:
        raise RuntimeError("final_sync_df has no recognizable arena frame column.")

    if "ms_axis" not in fs.columns:
        joined = False
        for eye_df in [getattr(bs, "left_eye_data", None), getattr(bs, "right_eye_data", None)]:
            if eye_df is not None and "OE_timestamp" in fs.columns and "OE_timestamp" in eye_df.columns and "ms_axis" in eye_df.columns:
                tmp = eye_df[["OE_timestamp", "ms_axis"]].dropna().drop_duplicates(subset=["OE_timestamp"])
                fs = fs.merge(tmp, on="OE_timestamp", how="left")
                joined = True
                break
        if not joined:
            fs["ms_axis"] = np.nan

    out = fs[[c for c in ["ms_axis", fcol, "OE_timestamp"] if c in fs.columns]].copy()
    out[fcol] = pd.to_numeric(out[fcol], errors="coerce").astype("Int64")
    out = out.dropna(subset=[fcol])
    return out, fcol

def export_curated_monocular_mp4(
    block_dict,
    curated_df,
    out_mp4_path,
    animal_col="animal",
    block_col="block",
    eye_col="eye",
    start_ms_col=None,   # auto-detect from start_ms / saccade_on_ms
    end_ms_col=None,     # auto-detect from end_ms / saccade_off_ms
    keep_col=None,       # auto-detect (keep_for_video / manual_outlier_detected / etc.)
    keep_value=True,     # include rows where keep_col == keep_value (after coercion)
    pre_ms=150,
    post_ms=150,
    fps_out=60.0,
    step_ms=None,        # default: 1000/fps_out
    window_scale=0.60,   # montage output scaling (controls final MP4 resolution)
    include_arena=False,
    arena_index=0,
    flip_mode="vertical",  # matches your viewer: "vertical" => cv2.flip(img,0):contentReference[oaicite:3]{index=3}
    font_scale=0.6,
    thickness=2,
    add_title_frames=True,
    title_frames_sec=0.5
):
    df = curated_df.copy().reset_index(drop=True)

    # --- resolve time columns (you noted you needed saccade_on/off in the search) ---
    cand_start = [start_ms_col, "start_ms", "saccade_on_ms"]
    cand_end   = [end_ms_col, "end_ms", "saccade_off_ms"]
    s_col = next((c for c in cand_start if c and c in df.columns), None)
    e_col = next((c for c in cand_end   if c and c in df.columns), None)
    if s_col is None or e_col is None:
        raise ValueError("Could not resolve start/end ms columns from {} / {}".format(cand_start, cand_end))

    # --- resolve keep column ---
    if keep_col is None:
        for c in ("keep_for_video", "keep", "include", "manual_outlier_detected", "is_good", "good", "tag"):
            if c in df.columns:
                keep_col = c
                break
    if keep_col is None:
        raise ValueError("Could not auto-detect keep_col. Pass keep_col explicitly.")

    def _coerce_bool(x):
        if x is None or (isinstance(x, float) and np.isnan(x)):
            return None
        if isinstance(x, bool):
            return x
        s = str(x).strip().lower()
        if s in ("true", "t", "1", "yes", "y"):
            return True
        if s in ("false", "f", "0", "no", "n"):
            return False
        return None

    df["_keep_bool"] = df[keep_col].map(_coerce_bool)

    # If keep_col is "manual_outlier_detected", typical meaning is BAD=True; invert by default if user wants "good"
    if keep_col == "manual_outlier_detected" and keep_value is True:
        # keep_value=True would keep "bad" — probably not what you want, so warn via behavior:
        # (we won't print; just recommend passing keep_value=False in usage)
        pass

    kept = df[df["_keep_bool"] == bool(keep_value)].copy()
    if kept.empty:
        raise ValueError("No rows matched keep_col={} == {} (after coercion).".format(keep_col, keep_value))

    # sort for reproducibility
    kept[block_col] = kept[block_col].astype(str).apply(_zfill_block)
    kept[s_col] = pd.to_numeric(kept[s_col], errors="coerce")
    kept[e_col] = pd.to_numeric(kept[e_col], errors="coerce")
    kept = kept.dropna(subset=[s_col, e_col])
    kept = kept.sort_values([animal_col, block_col, s_col]).reset_index(drop=True)

    if step_ms is None:
        step_ms = 1000.0 / float(fps_out)

    out_mp4_path = Path(out_mp4_path)
    out_mp4_path.parent.mkdir(parents=True, exist_ok=True)

    # --- helpers ---
    def _apply_flip(img):
        if flip_mode == "vertical":
            return cv2.flip(img, 0)
        return img

    def _overlay_text(img, lines, origin=(10, 24), vstep=22):
        x, y = origin
        for ln in lines:
            cv2.putText(img, str(ln), (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                        float(font_scale), (255, 255, 255), int(thickness), cv2.LINE_AA)
            y += vstep

    # --- open-on-demand per (animal, block) ---
    capL = capR = capA = None
    left_df = right_df = arena_df = None
    left_frame_col = right_frame_col = arena_frame_col = None
    cur_animal = None
    cur_block = None

    def _release():
        nonlocal capL, capR, capA
        for c in (capL, capR, capA):
            try:
                if c is not None:
                    c.release()
            except Exception:
                pass
        capL = capR = capA = None

    def _open_for(animal, block_num):
        nonlocal capL, capR, capA
        nonlocal left_df, right_df, arena_df
        nonlocal left_frame_col, right_frame_col, arena_frame_col
        nonlocal cur_animal, cur_block

        if animal == cur_animal and block_num == cur_block:
            return

        _release()
        bs = _block_for_key(block_dict, animal, block_num)
        cur_animal, cur_block = animal, block_num

        left_df = getattr(bs, "left_eye_data", None)
        right_df = getattr(bs, "right_eye_data", None)
        if left_df is None or right_df is None:
            raise RuntimeError("Missing eye data for {} B{}".format(animal, block_num))
        if "ms_axis" not in left_df.columns or "ms_axis" not in right_df.columns:
            raise RuntimeError("'ms_axis' missing for {} B{}".format(animal, block_num))

        left_frame_col = _frame_col(left_df)
        right_frame_col = _frame_col(right_df)
        if left_frame_col is None or right_frame_col is None:
            raise RuntimeError("Could not resolve eye frame column for {} B{}".format(animal, block_num))

        lv = Path(bs.le_videos[0])
        rv = Path(bs.re_videos[0])
        capL = cv2.VideoCapture(str(lv))
        capR = cv2.VideoCapture(str(rv))
        if not capL.isOpened():
            raise RuntimeError("Cannot open {}".format(lv))
        if not capR.isOpened():
            raise RuntimeError("Cannot open {}".format(rv))

        # arena optional
        capA = None
        arena_df = None
        arena_frame_col = None
        if include_arena and getattr(bs, "arena_videos", None):
            arena_df, arena_frame_col = _ensure_arena_ms_axis(bs)
            av = Path(bs.arena_videos[int(arena_index)])
            capA = cv2.VideoCapture(str(av))
            if not capA.isOpened():
                capA = None

    # --- determine output size using first event ---
    first = kept.iloc[0]
    a0 = str(first[animal_col])
    b0 = _zfill_block(first[block_col])
    _open_for(a0, b0)

    # grab one representative frame from each stream at start time
    ms0 = float(first[s_col])
    rowL = _nearest_row(left_df, ms0)
    rowR = _nearest_row(right_df, ms0)

    def _read_frame(cap, frame_idx):
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(max(0, frame_idx)))
        ret, f = cap.read()
        return f if ret else None

    L_idx = int(rowL[left_frame_col]) if rowL is not None and pd.notna(rowL[left_frame_col]) else 0
    R_idx = int(rowR[right_frame_col]) if rowR is not None and pd.notna(rowR[right_frame_col]) else 0
    fL = _read_frame(capL, L_idx)
    fR = _read_frame(capR, R_idx)
    if fL is None or fR is None:
        raise RuntimeError("Could not read initial frames for sizing.")

    fL = _apply_flip(fL)
    fR = _apply_flip(fR)

    # optional arena sizing
    if include_arena and capA is not None and arena_df is not None:
        arow = _nearest_row(arena_df, ms0)
        A_idx = int(arow[arena_frame_col]) if arow is not None and pd.notna(arow[arena_frame_col]) else 0
        fA = _read_frame(capA, A_idx)
        if fA is None:
            include_arena_eff = False
            fA = None
        else:
            include_arena_eff = True
            fA = _apply_flip(fA)
    else:
        include_arena_eff = False
        fA = None

    # build montage and scale
    def _montage(L_img, R_img, A_img=None):
        # make heights equal (use L height as reference)
        h = L_img.shape[0]
        if R_img.shape[0] != h:
            R_img = cv2.resize(R_img, (int(R_img.shape[1] * (h / float(R_img.shape[0]))), h))
        LR = cv2.hconcat([L_img, R_img])
        if A_img is not None:
            if A_img.shape[0] != h:
                A_img = cv2.resize(A_img, (int(A_img.shape[1] * (h / float(A_img.shape[0]))), h))
            LR = cv2.hconcat([LR, A_img])
        out = LR
        if window_scale is not None and float(window_scale) > 0 and float(window_scale) != 1.0:
            out = cv2.resize(out, (int(out.shape[1] * float(window_scale)), int(out.shape[0] * float(window_scale))))
        return out

    sample_m = _montage(fL, fR, fA if include_arena_eff else None)
    H_out, W_out = sample_m.shape[0], sample_m.shape[1]

    # --- open writer ---
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(out_mp4_path), fourcc, float(fps_out), (int(W_out), int(H_out)))
    if not writer.isOpened():
        raise RuntimeError("Could not open VideoWriter for {}".format(out_mp4_path))

    title_n = int(max(0, round(float(title_frames_sec) * float(fps_out))))

    # --- export loop ---
    try:
        for i in range(len(kept)):
            r = kept.iloc[i]
            animal = str(r[animal_col])
            block_num = _zfill_block(r[block_col])
            t0 = float(r[s_col]) - float(pre_ms)
            t1 = float(r[e_col]) + float(post_ms)

            _open_for(animal, block_num)

            # title card between events
            if add_title_frames and title_n > 0:
                card = np.zeros((H_out, W_out, 3), dtype=np.uint8)
                lines = [
                    "Monocular verification",
                    "Event {}/{}".format(i+1, len(kept)),
                    "{}  block {}".format(animal, block_num),
                    "t=[{:.1f}, {:.1f}] ms".format(float(r[s_col]), float(r[e_col])),
                ]
                _overlay_text(card, lines, origin=(20, 50), vstep=30)
                for _ in range(title_n):
                    writer.write(card)

            cur = t0
            while cur <= t1:
                rowL = _nearest_row(left_df, cur)
                rowR = _nearest_row(right_df, cur)

                # Left
                if rowL is not None and pd.notna(rowL[left_frame_col]):
                    L_idx = int(rowL[left_frame_col])
                    fL = _read_frame(capL, L_idx)
                    if fL is None:
                        fL = np.zeros((int(capL.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 1,
                                       int(capL.get(cv2.CAP_PROP_FRAME_WIDTH)) or 1, 3), dtype=np.uint8)
                else:
                    fL = np.zeros((int(capL.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 1,
                                   int(capL.get(cv2.CAP_PROP_FRAME_WIDTH)) or 1, 3), dtype=np.uint8)

                # Right
                if rowR is not None and pd.notna(rowR[right_frame_col]):
                    R_idx = int(rowR[right_frame_col])
                    fR = _read_frame(capR, R_idx)
                    if fR is None:
                        fR = np.zeros((int(capR.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 1,
                                       int(capR.get(cv2.CAP_PROP_FRAME_WIDTH)) or 1, 3), dtype=np.uint8)
                else:
                    fR = np.zeros((int(capR.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 1,
                                   int(capR.get(cv2.CAP_PROP_FRAME_WIDTH)) or 1, 3), dtype=np.uint8)

                fL = _apply_flip(fL)
                fR = _apply_flip(fR)

                # Arena (optional)
                fA = None
                if include_arena_eff and capA is not None and arena_df is not None:
                    arow = _nearest_row(arena_df, cur)
                    if arow is not None and pd.notna(arow[arena_frame_col]):
                        A_idx = int(arow[arena_frame_col])
                        fA = _read_frame(capA, A_idx)
                        if fA is not None:
                            fA = _apply_flip(fA)

                # overlays
                _overlay_text(fL, ["L  {} B{}  t={:.1f}ms".format(animal, block_num, cur)], origin=(10, 24))
                _overlay_text(fR, ["R  {} B{}  t={:.1f}ms".format(animal, block_num, cur)], origin=(10, 24))
                if fA is not None:
                    _overlay_text(fA, ["Arena  {} B{}  t={:.1f}ms".format(animal, block_num, cur)], origin=(10, 24))

                m = _montage(fL, fR, fA)
                # ensure exact size for writer
                if (m.shape[1], m.shape[0]) != (W_out, H_out):
                    m = cv2.resize(m, (W_out, H_out))
                writer.write(m)

                cur += float(step_ms)

    finally:
        writer.release()
        _release()

    print("Wrote:", out_mp4_path)
    return out_mp4_path


# -------------------------
# USAGE (after curation)
# -------------------------
# 1) If your curated df has 'keep_for_video' (True = include)
out_path = export_curated_monocular_mp4(block_dict, df, block.analysis_path / "monocular_verification.mp4",
                                        keep_col="keep_for_video", keep_value=True, window_scale=0.60)

# 2) If your curated df uses 'manual_outlier_detected' (True = reject/bad), include the GOOD ones:
# out_path = export_curated_monocular_mp4(block_dict, curated_df, r"Z:\...\monocular_verification.mp4",
#                                         keep_col="manual_outlier_detected", keep_value=False, window_scale=0.60)


Wrote: Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\analysis\monocular_verification.mp4


In [59]:
# ============================================================
# Export curated events to MP4 (no title cards + eye indicators + duration text + speed scalar)
# Python <3.9 compatible, stand-alone
# ============================================================
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

def export_curated_monocular_mp4_v2(
    block_dict,
    curated_df,
    out_mp4_path,
    # column names (auto-detected if None)
    animal_col="animal",
    block_col="block",
    eye_col="eye",
    start_ms_col=None,          # tries: start_ms, saccade_on_ms
    end_ms_col=None,            # tries: end_ms, saccade_off_ms
    keep_col=None,              # tries: keep_for_video, keep, include, manual_outlier_detected, ...
    keep_value=True,
    concurrent_col=None,        # optional: if provided and True -> show both circles
    # clip window around event
    pre_ms=150,
    post_ms=150,
    # output video timing
    fps_out=60.0,
    speed_scalar=1.0,           # 1=real-time, 0.5=half speed, 2=double speed (changes time step per output frame)
    # layout/appearance
    window_scale=0.60,
    include_arena=False,
    arena_index=0,
    flip_mode="vertical",       # "vertical" matches your older viewer (cv2.flip(img,0))
    circle_radius=10,
    circle_thickness=-1,        # -1 = filled
    circle_margin=18,           # px from top-left corner inside each eye panel
    text_corner="tl",           # "tl" or "tr"
    font_scale=0.6,
    thickness=2,
):
    """
    Writes one concatenated MP4 for all curated events where keep_col == keep_value.
    Changes from v1:
      - NO black title frames between events
      - green circle indicator for the saccading eye (both when concurrent)
      - corner text includes saccade duration (ms)
      - speed controlled by speed_scalar (time-step per output frame scaled by this value)
    """

    # ---------------- helpers ----------------
    def _zfill_block(block):
        s = str(block)
        digits = "".join([c for c in s if c.isdigit()])
        if digits == "":
            return s.zfill(3)
        return digits.zfill(3)

    def _block_for_key(block_dict_local, animal, block):
        b = _zfill_block(block)
        k1 = "{}_block_{}".format(str(animal), b)
        if k1 in block_dict_local:
            return block_dict_local[k1]
        if (str(animal), b) in block_dict_local:
            return block_dict_local[(str(animal), b)]
        raise KeyError("Could not find BlockSync for animal={} block={}".format(animal, b))

    def _frame_col(df_eye):
        for c in ("eye_frame", "frame", "frame_idx", "video_frame"):
            if c in df_eye.columns:
                return c
        return None

    def _nearest_row(df_eye, ms):
        arr = df_eye["ms_axis"].values
        if arr.size == 0:
            return None
        idx = int(np.argmin(np.abs(arr - float(ms))))
        return df_eye.iloc[idx]

    def _read_frame(cap, frame_idx):
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(max(0, frame_idx)))
        ret, f = cap.read()
        return f if ret else None

    def _apply_flip(img):
        if flip_mode == "vertical":
            return cv2.flip(img, 0)
        return img

    def _overlay_text(img, lines, origin=(10, 24), vstep=22):
        x, y = origin
        for ln in lines:
            cv2.putText(img, str(ln), (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                        float(font_scale), (255, 255, 255), int(thickness), cv2.LINE_AA)
            y += vstep

    def _coerce_bool(x):
        if x is None:
            return None
        try:
            if isinstance(x, float) and np.isnan(x):
                return None
        except Exception:
            pass
        if isinstance(x, bool):
            return x
        s = str(x).strip().lower()
        if s in ("true", "t", "1", "yes", "y"):
            return True
        if s in ("false", "f", "0", "no", "n"):
            return False
        return None

    def _detect_event_eyes(row):
        """
        Returns (left_active, right_active) for this event.
        - If concurrent_col is provided and True => both
        - Else infer from eye_col values
        """
        if concurrent_col is not None and concurrent_col in row.index:
            v = _coerce_bool(row[concurrent_col])
            if v is True:
                return True, True

        ev = str(row[eye_col]).strip().lower()
        if ev.startswith("l") or ev in ("left",):
            return True, False
        if ev.startswith("r") or ev in ("right",):
            return False, True
        # catch-all "both"/"sync"/"concurrent" style encodings
        if ("both" in ev) or ("sync" in ev) or ("concurrent" in ev) or ev in ("lr", "rl"):
            return True, True
        # default: unknown -> no circles
        return False, False

    def _put_circle(img, x, y):
        cv2.circle(img, (int(x), int(y)), int(circle_radius), (0, 255, 0), int(circle_thickness), cv2.LINE_AA)

    def _montage(L_img, R_img, A_img=None):
        # Equalize heights to L height
        h = L_img.shape[0]
        if R_img.shape[0] != h:
            R_img = cv2.resize(R_img, (int(R_img.shape[1] * (h / float(R_img.shape[0]))), h))
        LR = cv2.hconcat([L_img, R_img])
        if A_img is not None:
            if A_img.shape[0] != h:
                A_img = cv2.resize(A_img, (int(A_img.shape[1] * (h / float(A_img.shape[0]))), h))
            LR = cv2.hconcat([LR, A_img])
        out = LR
        if window_scale is not None and float(window_scale) > 0 and float(window_scale) != 1.0:
            out = cv2.resize(out, (int(out.shape[1] * float(window_scale)), int(out.shape[0] * float(window_scale))),
                             interpolation=cv2.INTER_AREA)
        return out

    def _ensure_arena_ms_axis(bs):
        """
        Uses bs.final_sync_df if it contains ms_axis, otherwise tries to merge ms_axis by OE_timestamp
        (same idea you used previously).
        """
        fs = bs.final_sync_df.copy()
        fcol = None
        for c in ["Arena_frame", "arena_frame", "frame", "frame_idx", "video_frame", "arena_idx"]:
            if c in fs.columns:
                fcol = c
                break
        if fcol is None:
            raise RuntimeError("final_sync_df has no recognizable arena frame column.")

        if "ms_axis" not in fs.columns:
            joined = False
            for eye_df in [getattr(bs, "left_eye_data", None), getattr(bs, "right_eye_data", None)]:
                if eye_df is not None and "OE_timestamp" in fs.columns and "OE_timestamp" in eye_df.columns and "ms_axis" in eye_df.columns:
                    tmp = eye_df[["OE_timestamp", "ms_axis"]].dropna().drop_duplicates(subset=["OE_timestamp"])
                    fs = fs.merge(tmp, on="OE_timestamp", how="left")
                    joined = True
                    break
            if not joined:
                fs["ms_axis"] = np.nan

        out = fs[[c for c in ["ms_axis", fcol, "OE_timestamp"] if c in fs.columns]].copy()
        out[fcol] = pd.to_numeric(out[fcol], errors="coerce").astype("Int64")
        out = out.dropna(subset=[fcol])
        return out, fcol

    # ---------------- input + resolve columns ----------------
    df = curated_df.copy().reset_index(drop=True)

    # start/end
    cand_start = [start_ms_col, "start_ms", "saccade_on_ms"]
    cand_end   = [end_ms_col, "end_ms", "saccade_off_ms"]
    s_col = next((c for c in cand_start if c and c in df.columns), None)
    e_col = next((c for c in cand_end   if c and c in df.columns), None)
    if s_col is None or e_col is None:
        raise ValueError("Could not resolve start/end columns from {} / {}".format(cand_start, cand_end))

    # keep col
    if keep_col is None:
        for c in ("keep_for_video", "keep", "include", "manual_outlier_detected", "is_good", "good", "tag"):
            if c in df.columns:
                keep_col = c
                break
    if keep_col is None:
        raise ValueError("Could not auto-detect keep_col. Pass keep_col explicitly.")

    # sanity
    for c in (animal_col, block_col, eye_col):
        if c not in df.columns:
            raise ValueError("Missing required column '{}' in curated_df.".format(c))

    df["_keep_bool"] = df[keep_col].map(_coerce_bool)

    kept = df[df["_keep_bool"] == bool(keep_value)].copy()
    if kept.empty:
        raise ValueError("No rows matched keep_col={} == {} (after coercion).".format(keep_col, keep_value))

    kept[block_col] = kept[block_col].astype(str).apply(_zfill_block)
    kept[s_col] = pd.to_numeric(kept[s_col], errors="coerce")
    kept[e_col] = pd.to_numeric(kept[e_col], errors="coerce")
    kept = kept.dropna(subset=[s_col, e_col]).sort_values([animal_col, block_col, s_col]).reset_index(drop=True)

    # speed scalar -> time step per output frame
    if speed_scalar is None:
        speed_scalar = 1.0
    speed_scalar = float(speed_scalar)
    if speed_scalar <= 0:
        raise ValueError("speed_scalar must be > 0. Got {}".format(speed_scalar))

    base_step_ms = 1000.0 / float(fps_out)
    step_ms = base_step_ms * speed_scalar  # <-- key behavior

    # output path
    out_mp4_path = Path(out_mp4_path)
    out_mp4_path.parent.mkdir(parents=True, exist_ok=True)

    # ---------------- open-on-demand per (animal, block) ----------------
    capL = capR = capA = None
    left_df = right_df = arena_df = None
    left_fc = right_fc = arena_fc = None
    cur_animal = None
    cur_block = None

    def _release():
        nonlocal capL, capR, capA
        for c in (capL, capR, capA):
            try:
                if c is not None:
                    c.release()
            except Exception:
                pass
        capL = capR = capA = None

    def _open_for(animal, block_num):
        nonlocal capL, capR, capA
        nonlocal left_df, right_df, arena_df
        nonlocal left_fc, right_fc, arena_fc
        nonlocal cur_animal, cur_block

        if (animal == cur_animal) and (block_num == cur_block):
            return

        _release()
        bs = _block_for_key(block_dict, animal, block_num)
        cur_animal, cur_block = animal, block_num

        left_df = getattr(bs, "left_eye_data", None)
        right_df = getattr(bs, "right_eye_data", None)
        if left_df is None or right_df is None:
            raise RuntimeError("Missing eye data for {} B{}".format(animal, block_num))
        if "ms_axis" not in left_df.columns or "ms_axis" not in right_df.columns:
            raise RuntimeError("'ms_axis' missing for {} B{}".format(animal, block_num))

        left_fc = _frame_col(left_df)
        right_fc = _frame_col(right_df)
        if left_fc is None or right_fc is None:
            raise RuntimeError("Could not resolve eye frame column for {} B{}".format(animal, block_num))

        lv = Path(bs.le_videos[0])
        rv = Path(bs.re_videos[0])
        capL = cv2.VideoCapture(str(lv))
        capR = cv2.VideoCapture(str(rv))
        if not capL.isOpened():
            raise RuntimeError("Cannot open {}".format(lv))
        if not capR.isOpened():
            raise RuntimeError("Cannot open {}".format(rv))

        # arena optional
        capA = None
        arena_df = None
        arena_fc = None
        if include_arena and getattr(bs, "arena_videos", None):
            if getattr(bs, "final_sync_df", None) is not None:
                arena_df, arena_fc = _ensure_arena_ms_axis(bs)
            av = Path(bs.arena_videos[int(arena_index)])
            capA = cv2.VideoCapture(str(av))
            if not capA.isOpened():
                capA = None

    # ---------------- determine output size from first event ----------------
    first = kept.iloc[0]
    a0 = str(first[animal_col])
    b0 = _zfill_block(first[block_col])
    _open_for(a0, b0)

    ms0 = float(first[s_col])
    rowL0 = _nearest_row(left_df, ms0)
    rowR0 = _nearest_row(right_df, ms0)
    L_idx0 = int(rowL0[left_fc]) if rowL0 is not None and pd.notna(rowL0[left_fc]) else 0
    R_idx0 = int(rowR0[right_fc]) if rowR0 is not None and pd.notna(rowR0[right_fc]) else 0
    fL0 = _read_frame(capL, L_idx0)
    fR0 = _read_frame(capR, R_idx0)
    if fL0 is None or fR0 is None:
        raise RuntimeError("Could not read initial frames for sizing.")
    fL0 = _apply_flip(fL0)
    fR0 = _apply_flip(fR0)

    fA0 = None
    include_arena_eff = False
    if include_arena and capA is not None and arena_df is not None and arena_fc is not None:
        arow0 = _nearest_row(arena_df, ms0)
        if arow0 is not None and pd.notna(arow0[arena_fc]):
            A_idx0 = int(arow0[arena_fc])
            fA0 = _read_frame(capA, A_idx0)
            if fA0 is not None:
                fA0 = _apply_flip(fA0)
                include_arena_eff = True

    sample = _montage(fL0, fR0, fA0 if include_arena_eff else None)
    H_out, W_out = sample.shape[0], sample.shape[1]

    # ---------------- open writer ----------------
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(out_mp4_path), fourcc, float(fps_out), (int(W_out), int(H_out)))
    if not writer.isOpened():
        raise RuntimeError("Could not open VideoWriter for {}".format(out_mp4_path))

    # corner text placement
    def _corner_origin(img, lines_count):
        h, w = img.shape[:2]
        if text_corner.lower() == "tr":
            # approximate width; keep margin
            return (max(10, w - 420), 24)
        return (10, 24)

    # ---------------- export loop ----------------
    try:
        for i in range(len(kept)):
            r = kept.iloc[i]
            animal = str(r[animal_col])
            block_num = _zfill_block(r[block_col])

            on_ms = float(r[s_col])
            off_ms = float(r[e_col])
            dur_ms = max(0.0, off_ms - on_ms)

            win_start = on_ms - float(pre_ms)
            win_end   = off_ms + float(post_ms)

            left_active, right_active = _detect_event_eyes(r)

            _open_for(animal, block_num)

            cur = win_start
            while cur <= win_end:
                # nearest frames by ms_axis
                rowL = _nearest_row(left_df, cur)
                rowR = _nearest_row(right_df, cur)

                # left frame
                if rowL is not None and pd.notna(rowL[left_fc]):
                    L_idx = int(rowL[left_fc])
                    fL = _read_frame(capL, L_idx)
                    if fL is None:
                        fL = np.zeros((int(capL.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 1,
                                       int(capL.get(cv2.CAP_PROP_FRAME_WIDTH)) or 1, 3), dtype=np.uint8)
                else:
                    fL = np.zeros((int(capL.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 1,
                                   int(capL.get(cv2.CAP_PROP_FRAME_WIDTH)) or 1, 3), dtype=np.uint8)

                # right frame
                if rowR is not None and pd.notna(rowR[right_fc]):
                    R_idx = int(rowR[right_fc])
                    fR = _read_frame(capR, R_idx)
                    if fR is None:
                        fR = np.zeros((int(capR.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 1,
                                       int(capR.get(cv2.CAP_PROP_FRAME_WIDTH)) or 1, 3), dtype=np.uint8)
                else:
                    fR = np.zeros((int(capR.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 1,
                                   int(capR.get(cv2.CAP_PROP_FRAME_WIDTH)) or 1, 3), dtype=np.uint8)

                fL = _apply_flip(fL)
                fR = _apply_flip(fR)

                # arena optional
                fA = None
                if include_arena_eff and capA is not None and arena_df is not None and arena_fc is not None:
                    arow = _nearest_row(arena_df, cur)
                    if arow is not None and pd.notna(arow[arena_fc]):
                        A_idx = int(arow[arena_fc])
                        fA = _read_frame(capA, A_idx)
                        if fA is not None:
                            fA = _apply_flip(fA)

                # circles (draw in each eye panel before montage)
                if left_active:
                    _put_circle(fL, circle_margin, circle_margin)
                if right_active:
                    _put_circle(fR, circle_margin, circle_margin)

                # light per-panel labels (optional)
                _overlay_text(fL, ["L"], origin=(10, fL.shape[0] - 10), vstep=18)
                _overlay_text(fR, ["R"], origin=(10, fR.shape[0] - 10), vstep=18)
                if fA is not None:
                    _overlay_text(fA, ["Arena"], origin=(10, fA.shape[0] - 10), vstep=18)

                m = _montage(fL, fR, fA)

                # corner text on the final montage
                lines = [
                    "Event {}/{}  {}  block {}".format(i + 1, len(kept), animal, block_num),
                    "Saccade dur: {:.1f} ms".format(dur_ms),
                    "t = {:.1f} ms   speed x{:.2f}".format(cur, float(speed_scalar)),
                ]
                _overlay_text(m, lines, origin=_corner_origin(m, len(lines)), vstep=22)

                # ensure fixed writer size
                if (m.shape[1], m.shape[0]) != (W_out, H_out):
                    m = cv2.resize(m, (W_out, H_out), interpolation=cv2.INTER_AREA)

                writer.write(m)
                cur += float(step_ms)

    finally:
        writer.release()
        _release()

    print("Wrote:", out_mp4_path)
    return out_mp4_path


# -------------------------
# CALL
# -------------------------
out_path = r'Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\analysis\monocular_verification_slo.mp4'
out_path = export_curated_monocular_mp4_v2(
    block_dict=block_dict,
    curated_df=curated_df2,
    out_mp4_path=out_path,
    keep_col="keep_for_video",     # or "manual_outlier_detected"
    keep_value=True,               # if manual_outlier_detected marks BAD, use keep_value=False
    start_ms_col="saccade_on_ms",
    end_ms_col="saccade_off_ms",
    eye_col="eye",
    pre_ms=200,
    post_ms=200,
    fps_out=60.0,
    speed_scalar=0.2,              # 0.5 half speed, 2.0 double speed
    window_scale=1,
    include_arena=True,
    concurrent_col=None            # set if you have a boolean "is_concurrent" column
)


Wrote: Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\analysis\monocular_verification_slo.mp4


In [58]:
# --- Monocular verification: OpenCV manual curation UI (sterile) ---
# PATCH: show green dot DURING saccade interval to indicate which eye is "active"

import re
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime

def curate_events_for_monocular_video_opencv(
    block_dict,
    events_df,
    animal_call=None,
    block=None,
    pre_ms=200.0,
    post_ms=200.0,
    flip_eyes=True,
    show_arena=False,
    text_cols=None,
    # display scaling:
    window_scale=0.60,
    scale_step=0.10,
    min_scale=0.20,
    max_scale=2.00,
    # playback:
    wait_ms_default=15,
    wait_step=5,
    min_wait_ms=1,
    max_wait_ms=200,
    # tagging (sterile)
    tag_col="keep_for_video",        # True=keep, False=reject, None/unset
    ts_col="curation_ts",
    out_dir_name="monocular_video_curation",
    overwrite_existing=False,
    # pre-annotation from existing column(s)
    autodetect_existing_tags=True,
    existing_tag_cols=None,
    invert_tag_cols=None,
    # NEW: active-eye dot during saccade
    show_active_dot=True,
    active_dot_only_during_saccade=True,
    active_dot_radius=10,
    active_dot_margin=18,            # px from top-left corner
    concurrent_col=None              # optional boolean column: True => show dot on both eyes
):
    """
    Manual curation UI for monocular events.

    NEW: green dot appears during the saccade interval to denote the currently "active" eye.
         This helps catch blink/mislabeled events (dot should coincide with actual saccade motion).

    Keyboard:
      Space: play/pause
      [ / ]: prev / next event
      , / .: step -1 / +1 frame
      k: KEEP
      r: REJECT
      e: export sterile per-block CSV(s)
      z / x: scale - / +
      - / +: slower / faster
      q / Esc: quit
    """

    if text_cols is None:
        text_cols = []

    if existing_tag_cols is None:
        existing_tag_cols = [
            "keep_for_video", "keep", "include", "include_in_monocular_video", "include_in_video",
            "is_good", "good", "accepted", "approved",
            "manual_outlier_detected", "outlier", "reject", "rejected", "is_bad", "bad"
        ]
    if invert_tag_cols is None:
        invert_tag_cols = set(["manual_outlier_detected", "outlier", "is_bad", "bad", "reject", "rejected"])
    else:
        invert_tag_cols = set(invert_tag_cols)

    # ---------------- helpers ----------------
    def _now_stamp():
        return datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    def _zfill_block(b):
        s = str(b)
        m = re.search(r"(\d+)", s)
        if m:
            return m.group(1).zfill(3)
        return s.zfill(3)

    def _first_existing(df, candidates):
        for c in candidates:
            if c in df.columns:
                return c
        return None

    def _require_col(df, candidates, label):
        c = _first_existing(df, candidates)
        if c is None:
            raise KeyError("Could not find {}. Tried: {}".format(label, candidates))
        return c

    def _coerce_tag(v):
        if v is None:
            return None
        try:
            if isinstance(v, float) and np.isnan(v):
                return None
        except Exception:
            pass
        if isinstance(v, (bool, np.bool_)):
            return bool(v)
        s = str(v).strip().lower()
        if s in ("true", "t", "1", "yes", "y", "keep", "include", "good", "accept"):
            return True
        if s in ("false", "f", "0", "no", "n", "reject", "bad", "exclude"):
            return False
        return None

    def _block_for_key(block_dict_local, animal, b):
        b = _zfill_block(b)
        animal = str(animal)

        if (animal, b) in block_dict_local:
            return block_dict_local[(animal, b)]

        k1 = "{}_block_{}".format(animal, b)
        if k1 in block_dict_local:
            return block_dict_local[k1]

        for k in block_dict_local.keys():
            if isinstance(k, str):
                if (animal in k) and ("block_{}".format(b) in k or k.endswith("_{}".format(b))):
                    return block_dict_local[k]

        raise KeyError("Could not find BlockSync for animal={} block={}".format(animal, b))

    def _get_eye_dfs(bs):
        left_df = getattr(bs, "left_eye_data", None)
        right_df = getattr(bs, "right_eye_data", None)
        if left_df is None:
            left_df = getattr(bs, "le_df", None)
        if right_df is None:
            right_df = getattr(bs, "re_df", None)
        return left_df, right_df

    def _get_video_paths(bs):
        lv = getattr(bs, "le_videos", None)
        rv = getattr(bs, "re_videos", None)
        av = getattr(bs, "arena_videos", None)

        lpath = None
        rpath = None
        apaths = []

        if isinstance(lv, (list, tuple)) and len(lv):
            lpath = lv[0]
        elif isinstance(lv, str):
            lpath = lv

        if isinstance(rv, (list, tuple)) and len(rv):
            rpath = rv[0]
        elif isinstance(rv, str):
            rpath = rv

        if isinstance(av, (list, tuple)):
            apaths = list(av)
        elif isinstance(av, str):
            apaths = [av]

        return lpath, rpath, apaths

    def _frame_col(df_eye):
        for c in ["eye_frame", "L_eye_frame", "R_eye_frame", "frame", "frame_idx"]:
            if c in df_eye.columns:
                return c
        return None

    def _nearest_row(df_eye, t_ms):
        if df_eye is None or df_eye.empty or "ms_axis" not in df_eye.columns:
            return None
        ms = df_eye["ms_axis"].values
        if ms.size == 0:
            return None
        idx = int(np.searchsorted(ms, t_ms))
        if idx <= 0:
            return df_eye.iloc[0]
        if idx >= ms.size:
            return df_eye.iloc[-1]
        if abs(ms[idx] - t_ms) < abs(ms[idx - 1] - t_ms):
            return df_eye.iloc[idx]
        return df_eye.iloc[idx - 1]

    def _median_step_ms(df_eye):
        if df_eye is None or df_eye.empty or "ms_axis" not in df_eye.columns:
            return 1000.0 / 60.0
        ms = df_eye["ms_axis"].values
        if ms.size < 2:
            return 1000.0 / 60.0
        d = np.diff(ms)
        d = d[np.isfinite(d) & (d > 0)]
        if d.size == 0:
            return 1000.0 / 60.0
        return float(np.median(d))

    def _apply_flip(img):
        if not flip_eyes:
            return img
        img = cv2.flip(img, 0)
        img = cv2.flip(img, 1)
        return img

    def _overlay_text(img, lines, origin=(10, 24), vstep=22, color=(255, 255, 255), font_scale=0.6, thickness=1):
        x, y = origin
        for ln in lines:
            cv2.putText(img, ln, (x, y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)
            y += vstep

    def _overlay_ellipse(img, df_eye, frame_idx, thickness=2):
        if df_eye is None or frame_idx is None:
            return
        fc = _frame_col(df_eye)
        if fc is None:
            return
        hit = df_eye[df_eye[fc] == frame_idx]
        if hit.empty:
            return
        row = hit.iloc[0]
        cx = row.get("center_x", np.nan)
        cy = row.get("center_y", np.nan)
        w  = row.get("width", np.nan)
        h  = row.get("height", np.nan)
        phi = row.get("phi", np.nan)
        if not (pd.isna(cx) or pd.isna(cy) or pd.isna(w) or pd.isna(h)):
            cv2.ellipse(
                img,
                (int(round(cx)), int(round(cy))),
                (max(1, int(round(w))), max(1, int(round(h)))),
                float(0 if pd.isna(phi) else phi),
                0, 360, (0, 255, 0),
                thickness
            )

    def _draw_active_dot(img):
        # draw AFTER flipping, so dot stays in the same visual corner
        cv2.circle(
            img,
            (int(active_dot_margin), int(active_dot_margin)),
            int(active_dot_radius),
            (0, 255, 0),
            -1,
            cv2.LINE_AA
        )

    def _event_eye_flags(event_eye_str, concurrent_flag):
        """
        Returns (left_active, right_active) based on event labels.
        """
        if concurrent_flag:
            return True, True

        s = str(event_eye_str).strip().lower()
        if s.startswith("l") or s == "left":
            return True, False
        if s.startswith("r") or s == "right":
            return False, True
        if ("both" in s) or ("sync" in s) or ("concurrent" in s) or s in ("lr", "rl"):
            return True, True
        return False, False

    def _ensure_std_columns(df_local, animal_col, block_col, eye_col, start_col, end_col):
        df_local["_animal_std"] = df_local[animal_col].astype(str)
        df_local["_block_std"]  = df_local[block_col].astype(str).apply(_zfill_block)
        df_local["_eye_std"]    = df_local[eye_col].astype(str)
        df_local["_start_ms_std"] = pd.to_numeric(df_local[start_col], errors="coerce")
        df_local["_end_ms_std"]   = pd.to_numeric(df_local[end_col], errors="coerce")
        df_local["_win_start_ms"] = df_local["_start_ms_std"] - float(pre_ms)
        df_local["_win_end_ms"]   = df_local["_end_ms_std"] + float(post_ms)
        if ts_col not in df_local.columns:
            df_local[ts_col] = ""
        return df_local

    def _write_block_annotations(bs, sub_df):
        out_dir = Path(bs.analysis_path) / out_dir_name
        out_dir.mkdir(parents=True, exist_ok=True)

        animal_s = str(sub_df["_animal_std"].iloc[0])
        block_s  = str(sub_df["_block_std"].iloc[0])
        out_path = out_dir / "{}_block_{}_monocular_curation.csv".format(animal_s, block_s)

        out = pd.DataFrame({
            "animal_call": sub_df["_animal_std"].astype(str),
            "block": sub_df["_block_std"].astype(str),
            "eye": sub_df["_eye_std"].astype(str),
            "start_ms": sub_df["_start_ms_std"].astype(float),
            "end_ms": sub_df["_end_ms_std"].astype(float),
            tag_col: sub_df[tag_col].map(_coerce_tag).astype("object"),
            ts_col: sub_df[ts_col].astype(str)
        })

        if out_path.exists() and (not overwrite_existing):
            try:
                old = pd.read_csv(out_path)
                key = ["animal_call", "block", "eye", "start_ms", "end_ms"]
                merged = old.merge(out, on=key, how="outer", suffixes=("_old", ""))
                if "{}_old".format(tag_col) in merged.columns:
                    merged[tag_col] = merged[tag_col].where(merged[tag_col].notna(), merged["{}_old".format(tag_col)])
                    merged = merged.drop(columns=["{}_old".format(tag_col)], errors="ignore")
                if "{}_old".format(ts_col) in merged.columns:
                    merged[ts_col] = merged[ts_col].where(merged[ts_col].astype(str) != "", merged["{}_old".format(ts_col)].astype(str))
                    merged = merged.drop(columns=["{}_old".format(ts_col)], errors="ignore")
                merged.to_csv(out_path, index=False)
                return out_path
            except Exception:
                out.to_csv(out_path, index=False)
                return out_path

        out.to_csv(out_path, index=False)
        return out_path

    # ---------------- normalize events df ----------------
    df = events_df.copy() if isinstance(events_df, pd.DataFrame) else pd.DataFrame(events_df).copy()

    animal_col = _require_col(df, ["animal_call", "animal", "_animal_std"], "animal column")
    block_col  = _require_col(df, ["block", "block_num", "_block_std"], "block column")
    eye_col    = _require_col(df, ["eye", "_eye_std"], "eye column")

    # prefer your names
    start_col = _require_col(df, ["_start_ms_std", "saccade_on_ms", "start_ms", "start_time_ms"], "start_ms column")
    end_col   = _require_col(df, ["_end_ms_std", "saccade_off_ms", "end_ms", "end_time_ms"], "end_ms column")

    df = _ensure_std_columns(df, animal_col, block_col, eye_col, start_col, end_col)
    df = df.dropna(subset=["_start_ms_std", "_end_ms_std"]).reset_index(drop=True)

    if animal_call is not None:
        df = df[df["_animal_std"].astype(str) == str(animal_call)].copy()
    if block is not None:
        df = df[df["_block_std"].astype(str) == _zfill_block(block)].copy()
    df = df.reset_index(drop=True)

    if df.empty:
        raise ValueError("No events to curate after filtering (animal_call/block).")

    # pre-annotate tags if requested
    if tag_col not in df.columns:
        df[tag_col] = None

    if autodetect_existing_tags:
        has_any = df[tag_col].map(_coerce_tag).notna().any()
        src_col = None
        if not has_any:
            for c in existing_tag_cols:
                if c == tag_col:
                    continue
                if c in df.columns and df[c].map(_coerce_tag).notna().any():
                    src_col = c
                    break
        if src_col is not None:
            src_is_inverted = (src_col in invert_tag_cols)
            cur = df[tag_col].map(_coerce_tag)
            src = df[src_col].map(_coerce_tag)
            if src_is_inverted:
                src = src.map(lambda v: (None if v is None else (not bool(v))))
            fill_mask = cur.isna() & src.notna()
            df.loc[fill_mask, tag_col] = src[fill_mask]
            if ts_col in df.columns:
                ts_empty = df[ts_col].astype(str).replace("nan", "").str.strip() == ""
                stamp_mask = fill_mask & ts_empty
                df.loc[stamp_mask, ts_col] = "preload_from:{}" .format(src_col)

    df[tag_col] = df[tag_col].map(_coerce_tag)

    # ---------------- OpenCV UI ----------------
    cv2.namedWindow("Controls", cv2.WINDOW_NORMAL)
    cv2.namedWindow("Left Eye", cv2.WINDOW_NORMAL)
    cv2.namedWindow("Right Eye", cv2.WINDOW_NORMAL)
    if show_arena:
        cv2.namedWindow("Arena", cv2.WINDOW_NORMAL)

    ctrl_w, ctrl_h = 540, 520
    buttons = {
        "Play":      ((10, 10),   (260, 60)),
        "Pause":     ((280, 10),  (530, 60)),
        "Prev":      ((10, 80),   (260, 130)),
        "Next":      ((280, 80),  (530, 130)),
        "Step -1":   ((10, 150),  (260, 200)),
        "Step +1":   ((280, 150), (530, 200)),
        "KEEP":      ((10, 220),  (260, 270)),
        "REJECT":    ((280, 220), (530, 270)),
        "Scale -":   ((10, 290),  (260, 340)),
        "Scale +":   ((280, 290), (530, 340)),
        "Slower":    ((10, 360),  (260, 410)),
        "Faster":    ((280, 360), (530, 410)),
        "Export":    ((10, 430),  (530, 480)),
    }

    COLOR_BORDER = (180, 180, 180)
    COLOR_TEXT = (220, 220, 220)
    COLOR_KEEP = (0, 255, 0)
    COLOR_REJ  = (0, 0, 255)
    COLOR_INFO = (180, 255, 180)
    COLOR_WARN = (0, 165, 255)

    playing = False
    quit_flag = False
    cur_idx = 0
    cur_ms = None
    step_ms = 1000.0 / 60.0
    wait_ms = int(wait_ms_default)
    last_status = ""

    capL = capR = capA = None
    current_key = (None, None)

    left_df = right_df = None
    left_fc = right_fc = None
    Wl = Hl = Wr = Hr = 0
    disp_Wl = disp_Hl = disp_Wr = disp_Hr = 0

    def _hit_button(x, y):
        for name, ((x1, y1), (x2, y2)) in buttons.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                return name
        return None

    def _format_state(v):
        v = _coerce_tag(v)
        if v is None:
            return "UNSET"
        return "KEEP" if v else "REJECT"

    def _draw_controls(idx):
        img = np.zeros((ctrl_h, ctrl_w, 3), dtype=np.uint8)
        state_str = _format_state(df[tag_col].iloc[idx])
        header = "Event {}/{} | state={} | scale={:.2f} | wait={}ms".format(
            idx + 1, len(df), state_str, float(window_scale), int(wait_ms)
        )
        cv2.putText(img, header, (10, ctrl_h - 12), cv2.FONT_HERSHEY_SIMPLEX, 0.55, COLOR_TEXT, 1, cv2.LINE_AA)
        if last_status:
            cv2.putText(img, last_status, (10, 22), cv2.FONT_HERSHEY_SIMPLEX, 0.55, COLOR_INFO, 1, cv2.LINE_AA)

        for name, ((x1, y1), (x2, y2)) in buttons.items():
            color = COLOR_BORDER
            if name == "KEEP":
                color = COLOR_KEEP
            elif name == "REJECT":
                color = COLOR_REJ
            elif name == "Export":
                color = COLOR_WARN
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, name, (x1 + 10, y2 - 18), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLOR_TEXT, 2, cv2.LINE_AA)
        return img

    def _seek(cap, idx):
        cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, int(idx)))

    def _read(cap):
        ret, f = cap.read()
        return f if ret else None

    def _set_display_scale():
        nonlocal disp_Wl, disp_Hl, disp_Wr, disp_Hr
        if Wl > 0 and Hl > 0:
            disp_Wl, disp_Hl = int(Wl * window_scale), int(Hl * window_scale)
            cv2.resizeWindow("Left Eye", disp_Wl, disp_Hl)
        if Wr > 0 and Hr > 0:
            disp_Wr, disp_Hr = int(Wr * window_scale), int(Hr * window_scale)
            cv2.resizeWindow("Right Eye", disp_Wr, disp_Hr)
        cv2.resizeWindow("Controls", ctrl_w, ctrl_h)

    def _open_for(animal, b):
        nonlocal capL, capR, capA, current_key
        nonlocal left_df, right_df, left_fc, right_fc, Wl, Hl, Wr, Hr

        key = (str(animal), _zfill_block(b))
        if key == current_key and capL is not None and capR is not None:
            return

        try:
            if capL is not None: capL.release()
            if capR is not None: capR.release()
            if capA is not None: capA.release()
        except Exception:
            pass

        bs = _block_for_key(block_dict, key[0], key[1])
        left_df, right_df = _get_eye_dfs(bs)
        left_fc = _frame_col(left_df) if left_df is not None else None
        right_fc = _frame_col(right_df) if right_df is not None else None

        lpath, rpath, _ = _get_video_paths(bs)
        if lpath is None or rpath is None:
            raise RuntimeError("Could not resolve le/re video paths for {} block {}".format(key[0], key[1]))

        capL = cv2.VideoCapture(str(lpath))
        capR = cv2.VideoCapture(str(rpath))
        if not capL.isOpened() or not capR.isOpened():
            raise RuntimeError("Could not open eye videos: L={} R={}".format(lpath, rpath))

        Wl = int(capL.get(cv2.CAP_PROP_FRAME_WIDTH))
        Hl = int(capL.get(cv2.CAP_PROP_FRAME_HEIGHT))
        Wr = int(capR.get(cv2.CAP_PROP_FRAME_WIDTH))
        Hr = int(capR.get(cv2.CAP_PROP_FRAME_HEIGHT))

        current_key = key
        _set_display_scale()

    def _export_per_block():
        nonlocal last_status
        written = []
        for (animal, b), sub in df.groupby(["_animal_std", "_block_std"], dropna=False):
            bs = _block_for_key(block_dict, animal, b)
            p = _write_block_annotations(bs, sub.copy())
            written.append(str(p))
        last_status = "Exported sterile curation to {} block file(s).".format(len(written))

    cv2.imshow("Controls", _draw_controls(cur_idx))

    def on_mouse_controls(event, x, y, flags, param):
        nonlocal playing, quit_flag, cur_idx, cur_ms, last_status, window_scale, wait_ms
        if event != cv2.EVENT_LBUTTONDOWN:
            return
        name = _hit_button(x, y)
        if name is None:
            return

        last_status = ""
        if name == "Play":
            playing = True
        elif name == "Pause":
            playing = False
        elif name == "Prev":
            playing = False
            cur_idx = (cur_idx - 1) % len(df)
            cur_ms = None
        elif name == "Next":
            playing = False
            cur_idx = (cur_idx + 1) % len(df)
            cur_ms = None
        elif name == "Step -1":
            playing = False
            if cur_ms is not None:
                cur_ms -= step_ms
        elif name == "Step +1":
            playing = False
            if cur_ms is not None:
                cur_ms += step_ms
        elif name == "KEEP":
            df.at[cur_idx, tag_col] = True
            df.at[cur_idx, ts_col] = _now_stamp()
        elif name == "REJECT":
            df.at[cur_idx, tag_col] = False
            df.at[cur_idx, ts_col] = _now_stamp()
        elif name == "Scale -":
            window_scale = max(min_scale, float(window_scale) - float(scale_step))
            _set_display_scale()
        elif name == "Scale +":
            window_scale = min(max_scale, float(window_scale) + float(scale_step))
            _set_display_scale()
        elif name == "Slower":
            wait_ms = min(max_wait_ms, int(wait_ms) + int(wait_step))
        elif name == "Faster":
            wait_ms = max(min_wait_ms, int(wait_ms) - int(wait_step))
        elif name == "Export":
            _export_per_block()

        cv2.imshow("Controls", _draw_controls(cur_idx))

    cv2.setMouseCallback("Controls", on_mouse_controls)

    # ---------------- main loop ----------------
    while True:
        k = cv2.waitKey(int(wait_ms)) & 0xFF

        if k in (27, ord('q'), ord('Q')):
            quit_flag = True
        elif k == 32:
            playing = not playing
        elif k == ord('['):
            playing = False
            cur_idx = (cur_idx - 1) % len(df)
            cur_ms = None
        elif k == ord(']'):
            playing = False
            cur_idx = (cur_idx + 1) % len(df)
            cur_ms = None
        elif k == ord(','):
            playing = False
            if cur_ms is not None:
                cur_ms -= step_ms
        elif k == ord('.'):
            playing = False
            if cur_ms is not None:
                cur_ms += step_ms
        elif k in (ord('k'), ord('K')):
            df.at[cur_idx, tag_col] = True
            df.at[cur_idx, ts_col] = _now_stamp()
        elif k in (ord('r'), ord('R')):
            df.at[cur_idx, tag_col] = False
            df.at[cur_idx, ts_col] = _now_stamp()
        elif k in (ord('e'), ord('E')):
            _export_per_block()
        elif k in (ord('z'), ord('Z')):
            window_scale = max(min_scale, float(window_scale) - float(scale_step))
            _set_display_scale()
        elif k in (ord('x'), ord('X')):
            window_scale = min(max_scale, float(window_scale) + float(scale_step))
            _set_display_scale()
        elif k == ord('-'):
            wait_ms = min(max_wait_ms, int(wait_ms) + int(wait_step))
        elif k in (ord('+'), ord('=')):
            wait_ms = max(min_wait_ms, int(wait_ms) - int(wait_step))

        if quit_flag:
            break

        row = df.iloc[cur_idx]
        animal = str(row["_animal_std"])
        b = str(row["_block_std"])

        _open_for(animal, b)
        step_ms = float(np.mean([_median_step_ms(left_df), _median_step_ms(right_df)]))

        win_start = float(row["_win_start_ms"])
        win_end   = float(row["_win_end_ms"])
        sac_on    = float(row["_start_ms_std"])
        sac_off   = float(row["_end_ms_std"])

        if cur_ms is None:
            cur_ms = win_start
        cur_ms = min(max(cur_ms, win_start), win_end)

        # determine which eyes should be “active” for THIS event
        concurrent_flag = False
        if concurrent_col is not None and concurrent_col in df.columns:
            v = _coerce_tag(row.get(concurrent_col, None))
            concurrent_flag = (v is True)

        left_active_flag, right_active_flag = _event_eye_flags(row["_eye_std"], concurrent_flag)

        # should the dot show at this timepoint?
        in_saccade = (sac_on <= cur_ms <= sac_off)
        show_dot_now = bool(show_active_dot) and (in_saccade if active_dot_only_during_saccade else True)

        rowL = _nearest_row(left_df, cur_ms)
        rowR = _nearest_row(right_df, cur_ms)

        # Left frame
        L_img = np.zeros((max(1, Hl), max(1, Wl), 3), dtype=np.uint8)
        if rowL is not None and left_fc is not None:
            L_idx = rowL.get(left_fc, np.nan)
            if pd.notna(L_idx):
                L_idx = int(L_idx)
                _seek(capL, L_idx)
                fL = _read(capL)
                if fL is not None:
                    _overlay_ellipse(fL, left_df, L_idx)
                    img = _apply_flip(fL.copy())

                    if show_dot_now and left_active_flag:
                        _draw_active_dot(img)

                    lines = ["Left | {} B{} | t={:.1f}ms | frame={}".format(animal, b, cur_ms, L_idx)]
                    if show_dot_now:
                        lines.append("saccade ACTIVE" if left_active_flag else "saccade inactive")
                    _overlay_text(img, lines, origin=(10, 24))
                    L_img = img
                else:
                    _overlay_text(L_img, ["Left | read error"], origin=(10, 24))
                    L_img = _apply_flip(L_img)
            else:
                _overlay_text(L_img, ["Left | no synchronized frame"], origin=(10, 24))
                L_img = _apply_flip(L_img)

        # Right frame
        R_img = np.zeros((max(1, Hr), max(1, Wr), 3), dtype=np.uint8)
        if rowR is not None and right_fc is not None:
            R_idx = rowR.get(right_fc, np.nan)
            if pd.notna(R_idx):
                R_idx = int(R_idx)
                _seek(capR, R_idx)
                fR = _read(capR)
                if fR is not None:
                    _overlay_ellipse(fR, right_df, R_idx)
                    img = _apply_flip(fR.copy())

                    if show_dot_now and right_active_flag:
                        _draw_active_dot(img)

                    lines = ["Right | {} B{} | t={:.1f}ms | frame={}".format(animal, b, cur_ms, R_idx)]
                    if show_dot_now:
                        lines.append("saccade ACTIVE" if right_active_flag else "saccade inactive")
                    _overlay_text(img, lines, origin=(10, 24))
                    R_img = img
                else:
                    _overlay_text(R_img, ["Right | read error"], origin=(10, 24))
                    R_img = _apply_flip(R_img)
            else:
                _overlay_text(R_img, ["Right | no synchronized frame"], origin=(10, 24))
                R_img = _apply_flip(R_img)

        # show
        if disp_Wl > 0 and disp_Hl > 0:
            cv2.imshow("Left Eye", cv2.resize(L_img, (disp_Wl, disp_Hl)))
        else:
            cv2.imshow("Left Eye", L_img)

        if disp_Wr > 0 and disp_Hr > 0:
            cv2.imshow("Right Eye", cv2.resize(R_img, (disp_Wr, disp_Hr)))
        else:
            cv2.imshow("Right Eye", R_img)

        cv2.imshow("Controls", _draw_controls(cur_idx))

        if playing:
            cur_ms += step_ms
            if cur_ms > win_end:
                playing = False
                cur_idx = (cur_idx + 1) % len(df)
                cur_ms = None

    try:
        if capL is not None: capL.release()
        if capR is not None: capR.release()
        if capA is not None: capA.release()
    except Exception:
        pass
    cv2.destroyAllWindows()

    return df


# -------------------------
# CALL EXAMPLE
# -------------------------
curated_df2 = curate_events_for_monocular_video_opencv(
    block_dict=block_dict,
    events_df=df,             # or an already curated df to edit
    animal_call="PV_106",
    block="011",
    tag_col="keep_for_video",
    show_active_dot=True,
    active_dot_only_during_saccade=True, # <-- key: dot appears only during on/off
    concurrent_col=None                  # set if you have an is_concurrent column
)
